# Creating a Sentiment Analysis Web App
## Using PyTorch and SageMaker

_Deep Learning Nanodegree Program | Deployment_

---

Now that we have a basic understanding of how SageMaker works we will try to use it to construct a complete project from end to end. Our goal will be to have a simple web page which a user can use to enter a movie review. The web page will then send the review off to our deployed model which will predict the sentiment of the entered review.

## Instructions

Some template code has already been provided for you, and you will need to implement additional functionality to successfully complete this notebook. You will not need to modify the included code beyond what is requested. Sections that begin with '**TODO**' in the header indicate that you need to complete or implement some portion within them. Instructions will be provided for each section and the specifics of the implementation are marked in the code block with a `# TODO: ...` comment. Please be sure to read the instructions carefully!

In addition to implementing code, there will be questions for you to answer which relate to the task and your implementation. Each section where you will answer a question is preceded by a '**Question:**' header. Carefully read each question and provide your answer below the '**Answer:**' header by editing the Markdown cell.

> **Note**: Code and Markdown cells can be executed using the **Shift+Enter** keyboard shortcut. In addition, a cell can be edited by typically clicking it (double-click for Markdown cells) or by pressing **Enter** while it is highlighted.

## General Outline

Recall the general outline for SageMaker projects using a notebook instance.

1. Download or otherwise retrieve the data.
2. Process / Prepare the data.
3. Upload the processed data to S3.
4. Train a chosen model.
5. Test the trained model (typically using a batch transform job).
6. Deploy the trained model.
7. Use the deployed model.

For this project, you will be following the steps in the general outline with some modifications. 

First, you will not be testing the model in its own step. You will still be testing the model, however, you will do it by deploying your model and then using the deployed model by sending the test data to it. One of the reasons for doing this is so that you can make sure that your deployed model is working correctly before moving forward.

In addition, you will deploy and use your trained model a second time. In the second iteration you will customize the way that your trained model is deployed by including some of your own code. In addition, your newly deployed model will be used in the sentiment analysis web app.

## Step 1: Downloading the data

As in the XGBoost in SageMaker notebook, we will be using the [IMDb dataset](http://ai.stanford.edu/~amaas/data/sentiment/)

> Maas, Andrew L., et al. [Learning Word Vectors for Sentiment Analysis](http://ai.stanford.edu/~amaas/data/sentiment/). In _Proceedings of the 49th Annual Meeting of the Association for Computational Linguistics: Human Language Technologies_. Association for Computational Linguistics, 2011.

In [1]:
%mkdir ../data
!wget -O ../data/aclImdb_v1.tar.gz http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar -zxf ../data/aclImdb_v1.tar.gz -C ../data

The syntax of the command is incorrect.
'wget' is not recognized as an internal or external command,
operable program or batch file.
tar: Error opening archive: Failed to open '../data/aclImdb_v1.tar.gz'


## Step 2: Preparing and Processing the data

Also, as in the XGBoost notebook, we will be doing some initial data processing. The first few steps are the same as in the XGBoost example. To begin with, we will read in each of the reviews and combine them into a single input structure. Then, we will split the dataset into a training set and a testing set.

In [2]:
import os
import glob

def read_imdb_data(data_dir='../data/aclImdb'):
    data = {}
    labels = {}
    
    for data_type in ['train', 'test']:
        data[data_type] = {}
        labels[data_type] = {}
        
        for sentiment in ['pos', 'neg']:
            data[data_type][sentiment] = []
            labels[data_type][sentiment] = []
            
            path = os.path.join(data_dir, data_type, sentiment, '*.txt')
            files = glob.glob(path)
            
            for f in files:
                with open(f) as review:
                    data[data_type][sentiment].append(review.read())
                    # Here we represent a positive review by '1' and a negative review by '0'
                    labels[data_type][sentiment].append(1 if sentiment == 'pos' else 0)
                    
            assert len(data[data_type][sentiment]) == len(labels[data_type][sentiment]), \
                    "{}/{} data size does not match labels size".format(data_type, sentiment)
                
    return data, labels

In [3]:
data, labels = read_imdb_data()
print("IMDB reviews: train = {} pos / {} neg, test = {} pos / {} neg".format(
            len(data['train']['pos']), len(data['train']['neg']),
            len(data['test']['pos']), len(data['test']['neg'])))

IMDB reviews: train = 12500 pos / 12500 neg, test = 12500 pos / 12500 neg


Now that we've read the raw training and testing data from the downloaded dataset, we will combine the positive and negative reviews and shuffle the resulting records.

In [5]:
from sklearn.utils import shuffle

def prepare_imdb_data(data, labels):
    """Prepare training and test sets from IMDb movie reviews."""
    
    #Combine positive and negative reviews and labels
    data_train = data['train']['pos'] + data['train']['neg']
    data_test = data['test']['pos'] + data['test']['neg']
    labels_train = labels['train']['pos'] + labels['train']['neg']
    labels_test = labels['test']['pos'] + labels['test']['neg']
    
    #Shuffle reviews and corresponding labels within training and test sets
    data_train, labels_train = shuffle(data_train, labels_train)
    data_test, labels_test = shuffle(data_test, labels_test)
    
    # Return a unified training data, test data, training labels, test labets
    return data_train, data_test, labels_train, labels_test

In [6]:
train_X, test_X, train_y, test_y = prepare_imdb_data(data, labels)
print("IMDb reviews (combined): train = {}, test = {}".format(len(train_X), len(test_X)))

IMDb reviews (combined): train = 25000, test = 25000


Now that we have our training and testing sets unified and prepared, we should do a quick check and see an example of the data our model will be trained on. This is generally a good idea as it allows you to see how each of the further processing steps affects the reviews and it also ensures that the data has been loaded correctly.

In [7]:
print(train_X[100])
print(train_y[100])

9, the film I've been looking forward to for months.... was little more then a disappointment.<br /><br />I was deeply surprised by 9's lack of story and strange character development. All the awesome action sequences in the world don't make up for a single unsympathetic character. <br /><br />The strange, almost thrown in occult sequences were not only out of place, they were infuriating. The story is about robots and scientists... why does it suddenly turn into a necronomicon horror wannabe with mystical symbols and green magic ghost lines instead of giving answers to what could have been excellently scary story devices??<br /><br />How, what, when, why.... questions that bode asking only if you care and it becomes less and less likely that you will as you get away from the theater. <br /><br />A film like this is frustrating because of its lack of depth.... I would watch this film drawn in crayon if the story was good. But the filmmakers have relied on CGI wizardry and Tim Burtons n

The first step in processing the reviews is to make sure that any html tags that appear should be removed. In addition we wish to tokenize our input, that way words such as *entertained* and *entertaining* are considered the same with regard to sentiment analysis.

In [8]:
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import *

import re
from bs4 import BeautifulSoup

def review_to_words(review):
    nltk.download("stopwords", quiet=True)
    stemmer = PorterStemmer()
    
    text = BeautifulSoup(review, "html.parser").get_text() # Remove HTML tags
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower()) # Convert to lower case
    words = text.split() # Split string into words
    words = [w for w in words if w not in stopwords.words("english")] # Remove stopwords
    words = [PorterStemmer().stem(w) for w in words] # stem
    
    return words

The `review_to_words` method defined above uses `BeautifulSoup` to remove any html tags that appear and uses the `nltk` package to tokenize the reviews. As a check to ensure we know how everything is working, try applying `review_to_words` to one of the reviews in the training set.

In [12]:
# TODO: Apply review_to_words to a review (train_X[100] or any other review)
print(review_to_words(train_X[100]))
# train_X[100] is an 'str'object
# the ouput of review_to_words is a list of words

['9', 'film', 'look', 'forward', 'month', 'littl', 'disappoint', 'deepli', 'surpris', '9', 'lack', 'stori', 'strang', 'charact', 'develop', 'awesom', 'action', 'sequenc', 'world', 'make', 'singl', 'unsympathet', 'charact', 'strang', 'almost', 'thrown', 'occult', 'sequenc', 'place', 'infuri', 'stori', 'robot', 'scientist', 'suddenli', 'turn', 'necronomicon', 'horror', 'wannab', 'mystic', 'symbol', 'green', 'magic', 'ghost', 'line', 'instead', 'give', 'answer', 'could', 'excel', 'scari', 'stori', 'devic', 'question', 'bode', 'ask', 'care', 'becom', 'less', 'less', 'like', 'get', 'away', 'theater', 'film', 'like', 'frustrat', 'lack', 'depth', 'would', 'watch', 'film', 'drawn', 'crayon', 'stori', 'good', 'filmmak', 'reli', 'cgi', 'wizardri', 'tim', 'burton', 'name', 'draw', 'crowd', 'drew', 'fail', 'gain', 'respect', '9', 'could', 'awesom', 'rewrit', 'littl', 'respect', 'creator']


**Question:** Above we mentioned that `review_to_words` method removes html formatting and allows us to tokenize the words found in a review, for example, converting *entertained* and *entertaining* into *entertain* so that they are treated as though they are the same word. What else, if anything, does this method do to the input?

**Answer:**
In addition to using nltk.stem to convert each word to the unique meaning it convey (so that we don't have many words that carry the same meaning). It does the following:
    1- converts the words to lower case
    2- uses regular expresssions (Regex) to replace any character that is not eitehr a letter or a number (such as an asterisks, dots, and commas) with a space
    3- extracte the words
    4- uses NLTK to remove stop words (words that don't identify the context such as a and an)

The method below applies the `review_to_words` method to each of the reviews in the training and testing datasets. In addition it caches the results. This is because performing this processing step can take a long time. This way if you are unable to complete the notebook in the current session, you can come back without needing to process the data a second time.

In [13]:
import pickle

cache_dir = os.path.join("../cache", "sentiment_analysis")  # where to store cache files
os.makedirs(cache_dir, exist_ok=True)  # ensure cache directory exists

def preprocess_data(data_train, data_test, labels_train, labels_test,
                    cache_dir=cache_dir, cache_file="preprocessed_data.pkl"):
    """Convert each review to words; read from cache if available."""

    # If cache_file is not None, try to read from it first
    cache_data = None
    if cache_file is not None:
        try:
            with open(os.path.join(cache_dir, cache_file), "rb") as f:
                cache_data = pickle.load(f)
            print("Read preprocessed data from cache file:", cache_file)
        except:
            pass  # unable to read from cache, but that's okay
    
    # If cache is missing, then do the heavy lifting
    if cache_data is None:
        # Preprocess training and test data to obtain words for each review
        #words_train = list(map(review_to_words, data_train))
        #words_test = list(map(review_to_words, data_test))
        words_train = [review_to_words(review) for review in data_train]
        words_test = [review_to_words(review) for review in data_test]
        
        # Write to cache file for future runs
        if cache_file is not None:
            cache_data = dict(words_train=words_train, words_test=words_test,
                              labels_train=labels_train, labels_test=labels_test)
            with open(os.path.join(cache_dir, cache_file), "wb") as f:
                pickle.dump(cache_data, f)
            print("Wrote preprocessed data to cache file:", cache_file)
    else:
        # Unpack data loaded from cache file
        words_train, words_test, labels_train, labels_test = (cache_data['words_train'],
                cache_data['words_test'], cache_data['labels_train'], cache_data['labels_test'])
    
    return words_train, words_test, labels_train, labels_test

In [14]:
# Preprocess data
train_X, test_X, train_y, test_y = preprocess_data(train_X, test_X, train_y, test_y)

Wrote preprocessed data to cache file: preprocessed_data.pkl


In [16]:
print(len(train_X))
print(train_X[100])

print(len(test_X))
print(test_X[100])

25000
['9', 'film', 'look', 'forward', 'month', 'littl', 'disappoint', 'deepli', 'surpris', '9', 'lack', 'stori', 'strang', 'charact', 'develop', 'awesom', 'action', 'sequenc', 'world', 'make', 'singl', 'unsympathet', 'charact', 'strang', 'almost', 'thrown', 'occult', 'sequenc', 'place', 'infuri', 'stori', 'robot', 'scientist', 'suddenli', 'turn', 'necronomicon', 'horror', 'wannab', 'mystic', 'symbol', 'green', 'magic', 'ghost', 'line', 'instead', 'give', 'answer', 'could', 'excel', 'scari', 'stori', 'devic', 'question', 'bode', 'ask', 'care', 'becom', 'less', 'less', 'like', 'get', 'away', 'theater', 'film', 'like', 'frustrat', 'lack', 'depth', 'would', 'watch', 'film', 'drawn', 'crayon', 'stori', 'good', 'filmmak', 'reli', 'cgi', 'wizardri', 'tim', 'burton', 'name', 'draw', 'crowd', 'drew', 'fail', 'gain', 'respect', '9', 'could', 'awesom', 'rewrit', 'littl', 'respect', 'creator']
25000
['american', 'werewolf', 'pari', 'realli', 'good', 'compar', 'origin', 'origin', 'use', 'comput', 

## Transform the data

In the XGBoost notebook we transformed the data from its word representation to a bag-of-words feature representation. For the model we are going to construct in this notebook we will construct a feature representation which is very similar. To start, we will represent each word as an integer. Of course, some of the words that appear in the reviews occur very infrequently and so likely don't contain much information for the purposes of sentiment analysis. The way we will deal with this problem is that we will fix the size of our working vocabulary and we will only include the words that appear most frequently. We will then combine all of the infrequent words into a single category and, in our case, we will label it as `1`.

Since we will be using a recurrent neural network, it will be convenient if the length of each review is the same. To do this, we will fix a size for our reviews and then pad short reviews with the category 'no word' (which we will label `0`) and truncate long reviews.

### (TODO) Create a word dictionary

To begin with, we need to construct a way to map words that appear in the reviews to integers. Here we fix the size of our vocabulary (including the 'no word' and 'infrequent' categories) to be `5000` but you may wish to change this to see how it affects the model.

> **TODO:** Complete the implementation for the `build_dict()` method below. Note that even though the vocab_size is set to `5000`, we only want to construct a mapping for the most frequently appearing `4998` words. This is because we want to reserve the special labels `0` for 'no word' and `1` for 'infrequent word'.

In [17]:
import numpy as np

def build_dict(data, vocab_size = 5000):
    """Construct and return a dictionary mapping each of the most frequently appearing words to a unique integer."""
    
    # TODO: Determine how often each word appears in `data`. Note that `data` is a list of sentences and that a
    #       sentence is a list of words.
    
    word_count = {} # A dict storing the words that appear in the reviews along with how often they occur
    for words_list in data:
        for word in words_list:
            if word in word_count:
                word_count[word] += 1
            else:
                word_count[word] = 1
            
    # TODO: Sort the words found in `data` so that sorted_words[0] is the most frequently appearing word and
    #       sorted_words[-1] is the least frequently appearing word.
    
    sorted_words = None
    sorted_words = sorted(word_count, key=word_count.get, reverse=True) # sort elements of teh dictionary according to the value of the <key, value> pair. the value = the number of times a word appeared
    # sorted_words is a list of the keys (or words) sorted according to the values (or number of occurences)
    
    word_dict = {} # This is what we are building, a dictionary that translates words into integers
    for idx, word in enumerate(sorted_words[:vocab_size - 2]): # The -2 is so that we save room for the 'no word'
        word_dict[word] = idx + 2                              # 'infrequent' labels
        
    return word_dict

In [18]:
word_dict = build_dict(train_X) # train_X is a list. An element in train_X is a list of words

In [24]:
print(len(word_dict))
print(word_dict)

4998
{'movi': 2, 'film': 3, 'one': 4, 'like': 5, 'time': 6, 'good': 7, 'make': 8, 'charact': 9, 'get': 10, 'see': 11, 'watch': 12, 'stori': 13, 'even': 14, 'would': 15, 'realli': 16, 'well': 17, 'scene': 18, 'look': 19, 'show': 20, 'much': 21, 'end': 22, 'peopl': 23, 'bad': 24, 'go': 25, 'great': 26, 'also': 27, 'first': 28, 'love': 29, 'think': 30, 'way': 31, 'act': 32, 'play': 33, 'made': 34, 'thing': 35, 'could': 36, 'know': 37, 'say': 38, 'seem': 39, 'work': 40, 'plot': 41, 'two': 42, 'actor': 43, 'year': 44, 'come': 45, 'mani': 46, 'seen': 47, 'take': 48, 'want': 49, 'life': 50, 'never': 51, 'littl': 52, 'best': 53, 'tri': 54, 'man': 55, 'ever': 56, 'give': 57, 'better': 58, 'still': 59, 'perform': 60, 'find': 61, 'feel': 62, 'part': 63, 'back': 64, 'use': 65, 'someth': 66, 'director': 67, 'actual': 68, 'interest': 69, 'lot': 70, 'real': 71, 'old': 72, 'cast': 73, 'though': 74, 'live': 75, 'star': 76, 'enjoy': 77, 'guy': 78, 'anoth': 79, 'new': 80, 'role': 81, 'noth': 82, '10': 83

**Question:** What are the five most frequently appearing (tokenized) words in the training set? Does it makes sense that these words appear frequently in the training set?

**Answer:** 'movi', 'film', 'one', 'like', 'time'.<br>
It makes sense that those words appear frequently but they are general words and don't convey the context. It would have been more descriptive if our vocabulary contained more words like 'bad', 'fail', 'horrible', impress' and less general words like those. 
<br>
<br>
It would have been better if we removed the words that appear whether the review is positive/negative. Like what we have did in the sentiment analysis class presented by Andrew Trask ( Lesson#6, Part#2 Neural Networks)

In [ ]:
# TODO: Use this space to determine the five most frequently appearing words in the training set.

### Save `word_dict`

Later on when we construct an endpoint which processes a submitted review we will need to make use of the `word_dict` which we have created. As such, we will save it to a file now for future use.

In [25]:
data_dir = '../data/pytorch' # The folder we will use for storing data
if not os.path.exists(data_dir): # Make sure that the folder exists
    os.makedirs(data_dir)

In [29]:
with open(os.path.join(data_dir, 'word_dict.pkl'), "wb") as f:
    pickle.dump(word_dict, f) # Write a pickled representation of obj to the open file object file.
    # binary file: A file object able to read and write bytes-like objects. Examples of binary files are files opened in binary mode ('rb', 'wb' or 'rb+')
    
# “Pickling” is the process whereby a Python object hierarchy is converted into a byte stream, and “unpickling” is the inverse operation, whereby a byte stream is converted back into an object hierarchy.
# Pickle data is opaque, binary data, even when you use protocol version 0
# I get "Error! word_dict.pkl is not UTF-8 encoded Saving disabled. See Console for more details."

In [2]:
word_dict = {'movi': 2, 'film': 3, 'one': 4, 'like': 5, 'time': 6, 'good': 7, 'make': 8, 'charact': 9, 'get': 10, 'see': 11, 'watch': 12, 'stori': 13, 'even': 14, 'would': 15, 'realli': 16, 'well': 17, 'scene': 18, 'look': 19, 'show': 20, 'much': 21, 'end': 22, 'peopl': 23, 'bad': 24, 'go': 25, 'great': 26, 'also': 27, 'first': 28, 'love': 29, 'think': 30, 'way': 31, 'act': 32, 'play': 33, 'made': 34, 'thing': 35, 'could': 36, 'know': 37, 'say': 38, 'seem': 39, 'work': 40, 'plot': 41, 'two': 42, 'actor': 43, 'year': 44, 'come': 45, 'mani': 46, 'seen': 47, 'take': 48, 'want': 49, 'life': 50, 'never': 51, 'littl': 52, 'best': 53, 'tri': 54, 'man': 55, 'ever': 56, 'give': 57, 'better': 58, 'still': 59, 'perform': 60, 'find': 61, 'feel': 62, 'part': 63, 'back': 64, 'use': 65, 'someth': 66, 'director': 67, 'actual': 68, 'interest': 69, 'lot': 70, 'real': 71, 'old': 72, 'cast': 73, 'though': 74, 'live': 75, 'star': 76, 'enjoy': 77, 'guy': 78, 'anoth': 79, 'new': 80, 'role': 81, 'noth': 82, '10': 83, 'funni': 84, 'music': 85, 'point': 86, 'start': 87, 'set': 88, 'girl': 89, 'origin': 90, 'day': 91, 'world': 92, 'everi': 93, 'believ': 94, 'turn': 95, 'quit': 96, 'us': 97, 'direct': 98, 'thought': 99, 'fact': 100, 'minut': 101, 'horror': 102, 'kill': 103, 'action': 104, 'comedi': 105, 'pretti': 106, 'young': 107, 'wonder': 108, 'happen': 109, 'around': 110, 'got': 111, 'effect': 112, 'right': 113, 'long': 114, 'howev': 115, 'big': 116, 'line': 117, 'famili': 118, 'enough': 119, 'seri': 120, 'may': 121, 'need': 122, 'fan': 123, 'bit': 124, 'script': 125, 'beauti': 126, 'person': 127, 'becom': 128, 'without': 129, 'must': 130, 'alway': 131, 'friend': 132, 'tell': 133, 'reason': 134, 'saw': 135, 'last': 136, 'final': 137, 'kid': 138, 'almost': 139, 'put': 140, 'least': 141, 'sure': 142, 'done': 143, 'whole': 144, 'place': 145, 'complet': 146, 'kind': 147, 'expect': 148, 'differ': 149, 'shot': 150, 'far': 151, 'mean': 152, 'anyth': 153, 'book': 154, 'laugh': 155, 'might': 156, 'name': 157, 'sinc': 158, 'begin': 159, '2': 160, 'probabl': 161, 'woman': 162, 'help': 163, 'entertain': 164, 'let': 165, 'screen': 166, 'call': 167, 'tv': 168, 'moment': 169, 'away': 170, 'read': 171, 'yet': 172, 'rather': 173, 'worst': 174, 'run': 175, 'fun': 176, 'lead': 177, 'hard': 178, 'audienc': 179, 'idea': 180, 'anyon': 181, 'episod': 182, 'american': 183, 'found': 184, 'appear': 185, 'bore': 186, 'especi': 187, 'although': 188, 'hope': 189, 'keep': 190, 'cours': 191, 'anim': 192, 'job': 193, 'goe': 194, 'move': 195, 'sens': 196, 'dvd': 197, 'version': 198, 'war': 199, 'money': 200, 'someon': 201, 'mind': 202, 'mayb': 203, 'problem': 204, 'true': 205, 'hous': 206, 'everyth': 207, 'nice': 208, 'second': 209, 'rate': 210, 'three': 211, 'night': 212, 'face': 213, 'follow': 214, 'recommend': 215, 'main': 216, 'product': 217, 'worth': 218, 'leav': 219, 'human': 220, 'special': 221, 'excel': 222, 'togeth': 223, 'wast': 224, 'everyon': 225, 'sound': 226, 'john': 227, 'hand': 228, '1': 229, 'father': 230, 'later': 231, 'eye': 232, 'said': 233, 'view': 234, 'instead': 235, 'review': 236, 'boy': 237, 'high': 238, 'hour': 239, 'miss': 240, 'classic': 241, 'talk': 242, 'wife': 243, 'understand': 244, 'left': 245, 'care': 246, 'black': 247, 'death': 248, 'open': 249, 'murder': 250, 'write': 251, 'half': 252, 'head': 253, 'rememb': 254, 'chang': 255, 'viewer': 256, 'fight': 257, 'gener': 258, 'surpris': 259, 'short': 260, 'includ': 261, 'die': 262, 'fall': 263, 'less': 264, 'els': 265, 'entir': 266, 'piec': 267, 'involv': 268, 'pictur': 269, 'simpli': 270, 'top': 271, 'home': 272, 'power': 273, 'total': 274, 'usual': 275, 'budget': 276, 'attempt': 277, 'suppos': 278, 'releas': 279, 'hollywood': 280, 'terribl': 281, 'song': 282, 'men': 283, 'possibl': 284, 'featur': 285, 'portray': 286, 'disappoint': 287, '3': 288, 'poor': 289, 'coupl': 290, 'stupid': 291, 'camera': 292, 'dead': 293, 'wrong': 294, 'low': 295, 'produc': 296, 'video': 297, 'either': 298, 'aw': 299, 'definit': 300, 'except': 301, 'rest': 302, 'given': 303, 'absolut': 304, 'women': 305, 'lack': 306, 'word': 307, 'writer': 308, 'titl': 309, 'talent': 310, 'decid': 311, 'full': 312, 'perfect': 313, 'along': 314, 'style': 315, 'close': 316, 'truli': 317, 'school': 318, 'emot': 319, 'save': 320, 'sex': 321, 'age': 322, 'next': 323, 'bring': 324, 'mr': 325, 'case': 326, 'killer': 327, 'heart': 328, 'comment': 329, 'sort': 330, 'creat': 331, 'perhap': 332, 'came': 333, 'brother': 334, 'sever': 335, 'joke': 336, 'art': 337, 'dialogu': 338, 'game': 339, 'small': 340, 'base': 341, 'flick': 342, 'written': 343, 'sequenc': 344, 'meet': 345, 'earli': 346, 'often': 347, 'other': 348, 'mother': 349, 'develop': 350, 'humor': 351, 'actress': 352, 'consid': 353, 'dark': 354, 'guess': 355, 'amaz': 356, 'unfortun': 357, 'lost': 358, 'light': 359, 'exampl': 360, 'cinema': 361, 'drama': 362, 'ye': 363, 'white': 364, 'experi': 365, 'imagin': 366, 'mention': 367, 'stop': 368, 'natur': 369, 'forc': 370, 'manag': 371, 'felt': 372, 'present': 373, 'cut': 374, 'children': 375, 'fail': 376, 'son': 377, 'qualiti': 378, 'support': 379, 'car': 380, 'ask': 381, 'hit': 382, 'side': 383, 'voic': 384, 'extrem': 385, 'impress': 386, 'evil': 387, 'wors': 388, 'stand': 389, 'went': 390, 'certainli': 391, 'basic': 392, 'oh': 393, 'overal': 394, 'favorit': 395, 'horribl': 396, 'mysteri': 397, 'number': 398, 'type': 399, 'danc': 400, 'wait': 401, 'hero': 402, '5': 403, 'alreadi': 404, 'learn': 405, 'matter': 406, '4': 407, 'michael': 408, 'genr': 409, 'fine': 410, 'despit': 411, 'throughout': 412, 'walk': 413, 'success': 414, 'histori': 415, 'question': 416, 'zombi': 417, 'town': 418, 'relationship': 419, 'realiz': 420, 'child': 421, 'past': 422, 'daughter': 423, 'late': 424, 'b': 425, 'wish': 426, 'hate': 427, 'credit': 428, 'event': 429, 'theme': 430, 'touch': 431, 'citi': 432, 'today': 433, 'sometim': 434, 'behind': 435, 'god': 436, 'twist': 437, 'sit': 438, 'deal': 439, 'stay': 440, 'annoy': 441, 'abl': 442, 'rent': 443, 'pleas': 444, 'edit': 445, 'blood': 446, 'deserv': 447, 'comic': 448, 'anyway': 449, 'appar': 450, 'soon': 451, 'gave': 452, 'etc': 453, 'level': 454, 'slow': 455, 'chanc': 456, 'score': 457, 'bodi': 458, 'brilliant': 459, 'incred': 460, 'figur': 461, 'situat': 462, 'self': 463, 'major': 464, 'stuff': 465, 'decent': 466, 'element': 467, 'dream': 468, 'return': 469, 'obvious': 470, 'continu': 471, 'order': 472, 'pace': 473, 'ridicul': 474, 'happi': 475, 'add': 476, 'highli': 477, 'group': 478, 'thank': 479, 'ladi': 480, 'novel': 481, 'pain': 482, 'speak': 483, 'career': 484, 'shoot': 485, 'strang': 486, 'heard': 487, 'sad': 488, 'polic': 489, 'husband': 490, 'import': 491, 'break': 492, 'took': 493, 'strong': 494, 'cannot': 495, 'predict': 496, 'robert': 497, 'violenc': 498, 'hilari': 499, 'recent': 500, 'countri': 501, 'known': 502, 'particularli': 503, 'pick': 504, 'documentari': 505, 'season': 506, 'critic': 507, 'jame': 508, 'compar': 509, 'obviou': 510, 'alon': 511, 'told': 512, 'state': 513, 'visual': 514, 'rock': 515, 'exist': 516, 'theater': 517, 'offer': 518, 'opinion': 519, 'gore': 520, 'hold': 521, 'crap': 522, 'result': 523, 'realiti': 524, 'hear': 525, 'room': 526, 'effort': 527, 'clich': 528, 'thriller': 529, 'caus': 530, 'sequel': 531, 'serious': 532, 'explain': 533, 'king': 534, 'local': 535, 'ago': 536, 'hell': 537, 'none': 538, 'note': 539, 'allow': 540, 'david': 541, 'sister': 542, 'simpl': 543, 'femal': 544, 'deliv': 545, 'ok': 546, 'class': 547, 'convinc': 548, 'check': 549, 'suspens': 550, 'win': 551, 'oscar': 552, 'buy': 553, 'huge': 554, 'valu': 555, 'sexual': 556, 'cool': 557, 'scari': 558, 'excit': 559, 'similar': 560, 'provid': 561, 'apart': 562, 'exactli': 563, 'avoid': 564, 'shown': 565, 'seriou': 566, 'english': 567, 'whose': 568, 'taken': 569, 'cinematographi': 570, 'shock': 571, 'polit': 572, 'spoiler': 573, 'offic': 574, 'across': 575, 'middl': 576, 'street': 577, 'pass': 578, 'messag': 579, 'somewhat': 580, 'silli': 581, 'charm': 582, 'modern': 583, 'filmmak': 584, 'confus': 585, 'form': 586, 'tale': 587, 'singl': 588, 'jack': 589, 'mostli': 590, 'attent': 591, 'william': 592, 'carri': 593, 'sing': 594, 'five': 595, 'subject': 596, 'richard': 597, 'prove': 598, 'stage': 599, 'team': 600, 'cop': 601, 'unlik': 602, 'georg': 603, 'televis': 604, 'monster': 605, 'earth': 606, 'cover': 607, 'villain': 608, 'pay': 609, 'marri': 610, 'toward': 611, 'build': 612, 'parent': 613, 'pull': 614, 'due': 615, 'respect': 616, 'fill': 617, 'dialog': 618, 'four': 619, 'remind': 620, 'futur': 621, 'typic': 622, 'weak': 623, '7': 624, 'cheap': 625, 'intellig': 626, 'atmospher': 627, 'british': 628, 'clearli': 629, '80': 630, 'non': 631, 'paul': 632, 'dog': 633, 'knew': 634, '8': 635, 'artist': 636, 'fast': 637, 'crime': 638, 'easili': 639, 'escap': 640, 'doubt': 641, 'adult': 642, 'detail': 643, 'date': 644, 'romant': 645, 'member': 646, 'fire': 647, 'drive': 648, 'gun': 649, 'straight': 650, 'beyond': 651, 'fit': 652, 'attack': 653, 'imag': 654, 'upon': 655, 'posit': 656, 'whether': 657, 'fantast': 658, 'peter': 659, 'appreci': 660, 'aspect': 661, 'captur': 662, 'ten': 663, 'plan': 664, 'discov': 665, 'remain': 666, 'near': 667, 'period': 668, 'air': 669, 'realist': 670, 'mark': 671, 'red': 672, 'dull': 673, 'adapt': 674, 'within': 675, 'lose': 676, 'spend': 677, 'materi': 678, 'color': 679, 'chase': 680, 'mari': 681, 'storylin': 682, 'forget': 683, 'bunch': 684, 'clear': 685, 'lee': 686, 'victim': 687, 'nearli': 688, 'box': 689, 'york': 690, 'inspir': 691, 'match': 692, 'mess': 693, 'finish': 694, 'standard': 695, 'easi': 696, 'truth': 697, 'busi': 698, 'suffer': 699, 'space': 700, 'dramat': 701, 'bill': 702, 'western': 703, 'e': 704, 'list': 705, 'battl': 706, 'notic': 707, 'de': 708, 'french': 709, 'ad': 710, '9': 711, 'tom': 712, 'larg': 713, 'among': 714, 'eventu': 715, 'train': 716, 'accept': 717, 'agre': 718, 'spirit': 719, 'soundtrack': 720, 'third': 721, 'teenag': 722, 'soldier': 723, 'adventur': 724, 'drug': 725, 'famou': 726, 'suggest': 727, 'sorri': 728, 'normal': 729, 'babi': 730, 'cri': 731, 'troubl': 732, 'ultim': 733, 'contain': 734, 'certain': 735, 'cultur': 736, 'romanc': 737, 'rare': 738, 'lame': 739, 'somehow': 740, 'mix': 741, 'disney': 742, 'gone': 743, 'cartoon': 744, 'student': 745, 'reveal': 746, 'fear': 747, 'suck': 748, 'kept': 749, 'attract': 750, 'appeal': 751, 'premis': 752, 'design': 753, 'secret': 754, 'greatest': 755, 'shame': 756, 'throw': 757, 'copi': 758, 'scare': 759, 'wit': 760, 'america': 761, 'admit': 762, 'brought': 763, 'particular': 764, 'relat': 765, 'screenplay': 766, 'whatev': 767, 'pure': 768, '70': 769, 'averag': 770, 'harri': 771, 'master': 772, 'describ': 773, 'male': 774, 'treat': 775, '20': 776, 'issu': 777, 'fantasi': 778, 'warn': 779, 'inde': 780, 'background': 781, 'forward': 782, 'free': 783, 'project': 784, 'japanes': 785, 'memor': 786, 'poorli': 787, 'award': 788, 'locat': 789, 'amus': 790, 'potenti': 791, 'struggl': 792, 'weird': 793, 'magic': 794, 'societi': 795, 'okay': 796, 'accent': 797, 'doctor': 798, 'imdb': 799, 'water': 800, 'hot': 801, 'express': 802, 'alien': 803, '30': 804, 'dr': 805, 'odd': 806, 'choic': 807, 'crazi': 808, 'fiction': 809, 'studio': 810, 'control': 811, 'becam': 812, 'masterpiec': 813, 'fli': 814, 'difficult': 815, 'joe': 816, 'scream': 817, 'costum': 818, 'lover': 819, 'uniqu': 820, 'refer': 821, 'remak': 822, 'girlfriend': 823, 'vampir': 824, 'prison': 825, 'execut': 826, 'wear': 827, 'jump': 828, 'unless': 829, 'wood': 830, 'creepi': 831, 'cheesi': 832, 'superb': 833, 'otherwis': 834, 'parti': 835, 'roll': 836, 'ghost': 837, 'public': 838, 'mad': 839, 'depict': 840, 'moral': 841, 'week': 842, 'earlier': 843, 'badli': 844, 'jane': 845, 'dumb': 846, 'fi': 847, 'flaw': 848, 'grow': 849, 'deep': 850, 'sci': 851, 'cat': 852, 'maker': 853, 'footag': 854, 'connect': 855, 'older': 856, 'bother': 857, 'plenti': 858, 'outsid': 859, 'stick': 860, 'gay': 861, 'catch': 862, 'plu': 863, 'co': 864, 'popular': 865, 'equal': 866, 'social': 867, 'quickli': 868, 'disturb': 869, 'perfectli': 870, 'dress': 871, '90': 872, 'era': 873, 'mistak': 874, 'lie': 875, 'previou': 876, 'ride': 877, 'combin': 878, 'concept': 879, 'band': 880, 'answer': 881, 'surviv': 882, 'rich': 883, 'front': 884, 'christma': 885, 'sweet': 886, 'insid': 887, 'eat': 888, 'bare': 889, 'concern': 890, 'beat': 891, 'listen': 892, 'ben': 893, 'c': 894, 'serv': 895, 'term': 896, 'german': 897, 'la': 898, 'meant': 899, 'stereotyp': 900, 'hardli': 901, 'innoc': 902, 'law': 903, 'desper': 904, 'memori': 905, 'promis': 906, 'intent': 907, 'cute': 908, 'variou': 909, 'steal': 910, 'inform': 911, 'brain': 912, 'post': 913, 'tone': 914, 'island': 915, 'amount': 916, 'nuditi': 917, 'compani': 918, 'track': 919, 'store': 920, 'claim': 921, 'flat': 922, 'hair': 923, '50': 924, 'land': 925, 'univers': 926, 'scott': 927, 'danger': 928, 'kick': 929, 'fairli': 930, 'player': 931, 'plain': 932, 'step': 933, 'crew': 934, 'toni': 935, 'share': 936, 'tast': 937, 'centuri': 938, 'achiev': 939, 'engag': 940, 'cold': 941, 'travel': 942, 'rip': 943, 'suit': 944, 'record': 945, 'sadli': 946, 'manner': 947, 'tension': 948, 'wrote': 949, 'spot': 950, 'fascin': 951, 'intens': 952, 'familiar': 953, 'burn': 954, 'remark': 955, 'depth': 956, 'histor': 957, 'destroy': 958, 'sleep': 959, 'purpos': 960, 'languag': 961, 'ignor': 962, 'ruin': 963, 'delight': 964, 'italian': 965, 'unbeliev': 966, 'abil': 967, 'collect': 968, 'soul': 969, 'detect': 970, 'clever': 971, 'violent': 972, 'rape': 973, 'reach': 974, 'door': 975, 'scienc': 976, 'trash': 977, 'liter': 978, 'reveng': 979, 'commun': 980, 'caught': 981, 'creatur': 982, 'approach': 983, 'trip': 984, 'fashion': 985, 'intrigu': 986, 'introduc': 987, 'paint': 988, 'skill': 989, 'complex': 990, 'channel': 991, 'camp': 992, 'christian': 993, 'hole': 994, 'extra': 995, 'mental': 996, 'immedi': 997, 'limit': 998, 'ann': 999, 'mere': 1000, 'million': 1001, '6': 1002, 'slightli': 1003, 'comput': 1004, 'conclus': 1005, 'slasher': 1006, 'suddenli': 1007, 'imposs': 1008, 'neither': 1009, 'crimin': 1010, 'teen': 1011, 'nation': 1012, 'physic': 1013, 'spent': 1014, 'respons': 1015, 'planet': 1016, 'receiv': 1017, 'fake': 1018, 'sick': 1019, 'blue': 1020, 'bizarr': 1021, 'embarrass': 1022, 'indian': 1023, 'ring': 1024, '15': 1025, 'drop': 1026, 'pop': 1027, 'drag': 1028, 'haunt': 1029, 'pointless': 1030, 'suspect': 1031, 'edg': 1032, 'search': 1033, 'handl': 1034, 'common': 1035, 'biggest': 1036, 'arriv': 1037, 'faith': 1038, 'hurt': 1039, 'technic': 1040, 'angel': 1041, 'genuin': 1042, 'dad': 1043, 'solid': 1044, 'f': 1045, 'awesom': 1046, 'van': 1047, 'focu': 1048, 'colleg': 1049, 'former': 1050, 'count': 1051, 'tear': 1052, 'heavi': 1053, 'rais': 1054, 'wall': 1055, 'younger': 1056, 'visit': 1057, 'laughabl': 1058, 'excus': 1059, 'sign': 1060, 'fair': 1061, 'cult': 1062, 'tough': 1063, 'motion': 1064, 'key': 1065, 'desir': 1066, 'super': 1067, 'addit': 1068, 'stun': 1069, 'exploit': 1070, 'cloth': 1071, 'smith': 1072, 'tortur': 1073, 'davi': 1074, 'race': 1075, 'author': 1076, 'cross': 1077, 'jim': 1078, 'focus': 1079, 'consist': 1080, 'compel': 1081, 'minor': 1082, 'chemistri': 1083, 'commit': 1084, 'pathet': 1085, 'park': 1086, 'tradit': 1087, 'frank': 1088, 'obsess': 1089, 'grade': 1090, 'asid': 1091, '60': 1092, 'brutal': 1093, 'somewher': 1094, 'steve': 1095, 'rule': 1096, 'grant': 1097, 'opportun': 1098, 'depress': 1099, 'u': 1100, 'explor': 1101, 'besid': 1102, 'honest': 1103, 'anti': 1104, 'dub': 1105, 'intend': 1106, 'trailer': 1107, 'bar': 1108, 'longer': 1109, 'scientist': 1110, 'west': 1111, 'regard': 1112, 'decad': 1113, 'judg': 1114, 'silent': 1115, 'creativ': 1116, 'armi': 1117, 'wild': 1118, 'g': 1119, 'south': 1120, 'stewart': 1121, 'draw': 1122, 'road': 1123, 'govern': 1124, 'boss': 1125, 'ex': 1126, 'practic': 1127, 'motiv': 1128, 'gang': 1129, 'festiv': 1130, 'club': 1131, 'surprisingli': 1132, 'page': 1133, 'london': 1134, 'redeem': 1135, 'green': 1136, 'machin': 1137, 'idiot': 1138, 'aliv': 1139, 'display': 1140, 'militari': 1141, 'thrill': 1142, 'repeat': 1143, '100': 1144, 'nobodi': 1145, 'folk': 1146, 'yeah': 1147, '40': 1148, 'journey': 1149, 'garbag': 1150, 'tire': 1151, 'smile': 1152, 'ground': 1153, 'mood': 1154, 'bought': 1155, 'cost': 1156, 'stone': 1157, 'sam': 1158, 'mouth': 1159, 'noir': 1160, 'terrif': 1161, 'agent': 1162, 'utterli': 1163, 'requir': 1164, 'honestli': 1165, 'sexi': 1166, 'area': 1167, 'geniu': 1168, 'report': 1169, 'glad': 1170, 'enter': 1171, 'humour': 1172, 'investig': 1173, 'serial': 1174, 'passion': 1175, 'narr': 1176, 'occasion': 1177, 'marriag': 1178, 'climax': 1179, 'industri': 1180, 'studi': 1181, 'center': 1182, 'charli': 1183, 'nowher': 1184, 'ship': 1185, 'demon': 1186, 'loos': 1187, 'bear': 1188, 'hors': 1189, 'wow': 1190, 'hang': 1191, 'graphic': 1192, 'giant': 1193, 'admir': 1194, 'send': 1195, 'damn': 1196, 'loud': 1197, 'rel': 1198, 'nake': 1199, 'profession': 1200, 'subtl': 1201, 'blow': 1202, 'bottom': 1203, 'insult': 1204, 'batman': 1205, 'doubl': 1206, 'kelli': 1207, 'boyfriend': 1208, 'r': 1209, 'initi': 1210, 'frame': 1211, 'gem': 1212, 'opera': 1213, 'church': 1214, 'challeng': 1215, 'drawn': 1216, 'cinemat': 1217, 'affect': 1218, 'nightmar': 1219, 'seek': 1220, 'l': 1221, 'fulli': 1222, 'j': 1223, 'evid': 1224, 'essenti': 1225, 'conflict': 1226, 'arm': 1227, 'christoph': 1228, 'wind': 1229, 'henri': 1230, 'grace': 1231, 'witch': 1232, 'assum': 1233, 'narrat': 1234, 'push': 1235, 'hunt': 1236, 'chri': 1237, 'wise': 1238, 'nomin': 1239, 'month': 1240, 'repres': 1241, 'sceneri': 1242, 'hide': 1243, 'affair': 1244, 'avail': 1245, 'smart': 1246, 'thu': 1247, 'justic': 1248, 'bond': 1249, 'flashback': 1250, 'interview': 1251, 'outstand': 1252, 'satisfi': 1253, 'constantli': 1254, 'presenc': 1255, 'bed': 1256, 'central': 1257, 'iron': 1258, 'content': 1259, 'sell': 1260, 'everybodi': 1261, 'gag': 1262, 'hotel': 1263, 'slowli': 1264, 'hire': 1265, 'system': 1266, 'adam': 1267, 'hey': 1268, 'individu': 1269, 'thrown': 1270, 'charl': 1271, 'mediocr': 1272, 'jone': 1273, 'allen': 1274, 'lesson': 1275, 'billi': 1276, 'ray': 1277, 'photographi': 1278, 'cameo': 1279, 'fellow': 1280, 'pari': 1281, 'strike': 1282, 'independ': 1283, 'rise': 1284, 'brief': 1285, 'absurd': 1286, 'neg': 1287, 'impact': 1288, 'phone': 1289, 'born': 1290, 'model': 1291, 'ill': 1292, 'fresh': 1293, 'angl': 1294, 'spoil': 1295, 'likabl': 1296, 'abus': 1297, 'discuss': 1298, 'hill': 1299, 'sight': 1300, 'ahead': 1301, 'sent': 1302, 'photograph': 1303, 'shine': 1304, 'blame': 1305, 'logic': 1306, 'occur': 1307, 'bruce': 1308, 'mainli': 1309, 'commerci': 1310, 'forev': 1311, 'skip': 1312, 'surround': 1313, 'held': 1314, 'teacher': 1315, 'segment': 1316, 'zero': 1317, 'blond': 1318, 'summer': 1319, 'resembl': 1320, 'trap': 1321, 'satir': 1322, 'six': 1323, 'queen': 1324, 'ball': 1325, 'fool': 1326, 'sub': 1327, 'tragedi': 1328, 'twice': 1329, 'reaction': 1330, 'pack': 1331, 'bomb': 1332, 'will': 1333, 'hospit': 1334, 'protagonist': 1335, 'sport': 1336, 'mile': 1337, 'trust': 1338, 'vote': 1339, 'mom': 1340, 'drink': 1341, 'jerri': 1342, 'encount': 1343, 'plane': 1344, 'al': 1345, 'current': 1346, 'station': 1347, 'program': 1348, 'celebr': 1349, 'martin': 1350, 'choos': 1351, 'join': 1352, 'field': 1353, 'tragic': 1354, 'lord': 1355, 'favourit': 1356, 'round': 1357, 'vision': 1358, 'jean': 1359, 'robot': 1360, 'arthur': 1361, 'tie': 1362, 'random': 1363, 'fortun': 1364, 'roger': 1365, 'dread': 1366, 'psycholog': 1367, 'intern': 1368, 'epic': 1369, 'nonsens': 1370, 'prefer': 1371, 'improv': 1372, 'legend': 1373, 'formula': 1374, 'highlight': 1375, 'pleasur': 1376, 'tape': 1377, 'dollar': 1378, '11': 1379, 'gorgeou': 1380, 'fox': 1381, 'porn': 1382, 'wide': 1383, 'thin': 1384, 'object': 1385, 'buddi': 1386, 'ugli': 1387, 'influenc': 1388, 'ii': 1389, 'prepar': 1390, 'nasti': 1391, 'reflect': 1392, 'supposedli': 1393, 'progress': 1394, 'warm': 1395, 'youth': 1396, 'worthi': 1397, 'latter': 1398, 'unusu': 1399, 'length': 1400, 'crash': 1401, 'superior': 1402, 'childhood': 1403, 'seven': 1404, 'shop': 1405, 'remot': 1406, 'theatr': 1407, 'disgust': 1408, 'funniest': 1409, 'pilot': 1410, 'paid': 1411, 'convers': 1412, 'castl': 1413, 'trick': 1414, 'fell': 1415, 'establish': 1416, 'gangster': 1417, 'disast': 1418, 'rob': 1419, 'ident': 1420, 'heaven': 1421, 'suicid': 1422, 'disappear': 1423, 'mine': 1424, 'decis': 1425, 'forgotten': 1426, 'singer': 1427, 'mask': 1428, 'tend': 1429, 'heroin': 1430, 'brian': 1431, 'partner': 1432, 'desert': 1433, 'recogn': 1434, 'alan': 1435, 'p': 1436, 'stuck': 1437, 'ms': 1438, 'sky': 1439, 'thoroughli': 1440, 'replac': 1441, 'accur': 1442, 'market': 1443, 'andi': 1444, 'danni': 1445, 'commentari': 1446, 'uncl': 1447, 'eddi': 1448, 'seemingli': 1449, 'clue': 1450, 'devil': 1451, 'jackson': 1452, 'that': 1453, 'refus': 1454, 'therefor': 1455, 'pair': 1456, 'unit': 1457, 'accid': 1458, 'ed': 1459, 'fault': 1460, 'river': 1461, 'fate': 1462, 'afraid': 1463, 'tune': 1464, 'russian': 1465, 'hidden': 1466, 'stephen': 1467, 'clean': 1468, 'irrit': 1469, 'convey': 1470, 'captain': 1471, 'readi': 1472, 'test': 1473, 'quick': 1474, 'instanc': 1475, 'european': 1476, 'daniel': 1477, 'frustrat': 1478, 'insan': 1479, '1950': 1480, 'chines': 1481, 'wed': 1482, 'rescu': 1483, 'food': 1484, 'dirti': 1485, 'angri': 1486, 'lock': 1487, 'joy': 1488, 'price': 1489, 'steven': 1490, 'bland': 1491, 'cage': 1492, 'anymor': 1493, 'rang': 1494, 'wooden': 1495, 'jason': 1496, 'rush': 1497, 'news': 1498, 'n': 1499, 'twenti': 1500, 'worri': 1501, 'martial': 1502, 'led': 1503, 'board': 1504, '12': 1505, 'cgi': 1506, 'transform': 1507, 'hunter': 1508, 'symbol': 1509, 'johnni': 1510, 'sentiment': 1511, 'piti': 1512, 'onto': 1513, 'x': 1514, 'invent': 1515, 'explan': 1516, 'process': 1517, 'attitud': 1518, 'owner': 1519, 'awar': 1520, 'aim': 1521, 'target': 1522, 'favor': 1523, 'energi': 1524, 'floor': 1525, 'necessari': 1526, 'religi': 1527, 'opposit': 1528, 'insight': 1529, 'window': 1530, 'chick': 1531, 'blind': 1532, 'movement': 1533, 'research': 1534, 'mountain': 1535, 'comparison': 1536, 'deepli': 1537, 'possess': 1538, 'grand': 1539, 'comed': 1540, 'whatsoev': 1541, 'rain': 1542, 'mid': 1543, 'began': 1544, 'shadow': 1545, 'bank': 1546, 'parodi': 1547, 'princ': 1548, 'credibl': 1549, 'taylor': 1550, 'weapon': 1551, 'friendship': 1552, 'pre': 1553, 'flesh': 1554, 'dougla': 1555, 'teach': 1556, 'terror': 1557, 'protect': 1558, 'hint': 1559, 'bloodi': 1560, 'marvel': 1561, 'leader': 1562, 'load': 1563, 'anybodi': 1564, 'drunk': 1565, 'accord': 1566, 'watchabl': 1567, 'superman': 1568, 'freddi': 1569, 'brown': 1570, 'tim': 1571, 'appropri': 1572, 'seat': 1573, 'jeff': 1574, 'hitler': 1575, 'charg': 1576, 'keaton': 1577, 'knock': 1578, 'villag': 1579, 'unknown': 1580, 'empti': 1581, 'unnecessari': 1582, 'enemi': 1583, 'england': 1584, 'media': 1585, 'strength': 1586, 'wave': 1587, 'buck': 1588, 'dare': 1589, 'perspect': 1590, 'utter': 1591, 'craft': 1592, 'kiss': 1593, 'contrast': 1594, 'ford': 1595, 'correct': 1596, 'nativ': 1597, 'chill': 1598, 'distract': 1599, 'speed': 1600, 'anywher': 1601, 'soap': 1602, 'knowledg': 1603, 'magnific': 1604, 'nazi': 1605, 'breath': 1606, 'mission': 1607, 'fred': 1608, '1980': 1609, 'ice': 1610, 'moon': 1611, 'crowd': 1612, 'jr': 1613, 'joan': 1614, 'kate': 1615, '000': 1616, 'soft': 1617, 'frighten': 1618, 'nick': 1619, 'dick': 1620, 'dan': 1621, 'hundr': 1622, 'radio': 1623, 'somebodi': 1624, 'dozen': 1625, 'simon': 1626, 'academi': 1627, 'andrew': 1628, 'thousand': 1629, 'shakespear': 1630, 'loss': 1631, 'vehicl': 1632, 'quot': 1633, 'sum': 1634, 'account': 1635, 'root': 1636, 'convent': 1637, 'behavior': 1638, 'leg': 1639, '1970': 1640, 'regular': 1641, 'gold': 1642, 'pretenti': 1643, 'worker': 1644, 'compet': 1645, 'demand': 1646, 'explos': 1647, 'stretch': 1648, 'candi': 1649, 'japan': 1650, 'notabl': 1651, 'privat': 1652, 'interpret': 1653, 'lynch': 1654, 'debut': 1655, 'tarzan': 1656, 'constant': 1657, 'translat': 1658, 'prais': 1659, 'revolv': 1660, 'sea': 1661, 'spi': 1662, 'sat': 1663, 'jesu': 1664, 'quiet': 1665, 'ass': 1666, 'threaten': 1667, 'failur': 1668, 'franc': 1669, 'technolog': 1670, 'aid': 1671, 'met': 1672, 'kevin': 1673, 'punch': 1674, 'toy': 1675, 'higher': 1676, 'vh': 1677, 'interact': 1678, 'abandon': 1679, 'mike': 1680, 'separ': 1681, 'confront': 1682, 'command': 1683, 'bet': 1684, 'gotten': 1685, 'servic': 1686, 'recal': 1687, 'stunt': 1688, 'belong': 1689, 'techniqu': 1690, 'site': 1691, 'foot': 1692, 'freak': 1693, 'cabl': 1694, 'bug': 1695, 'capabl': 1696, 'fu': 1697, 'african': 1698, 'bright': 1699, 'jimmi': 1700, 'fat': 1701, 'boat': 1702, 'presid': 1703, 'clark': 1704, 'succeed': 1705, 'stock': 1706, 'gene': 1707, 'structur': 1708, 'spanish': 1709, 'kidnap': 1710, 'paper': 1711, 'factor': 1712, 'belief': 1713, 'whilst': 1714, 'witti': 1715, 'attend': 1716, 'tree': 1717, 'realis': 1718, 'complic': 1719, 'educ': 1720, 'bob': 1721, 'realism': 1722, 'broken': 1723, 'finest': 1724, 'assist': 1725, 'santa': 1726, 'depart': 1727, 'smoke': 1728, 'determin': 1729, 'observ': 1730, 'up': 1731, 'v': 1732, 'domin': 1733, 'oper': 1734, 'fame': 1735, 'routin': 1736, 'lewi': 1737, 'hat': 1738, 'rubbish': 1739, 'advanc': 1740, 'morgan': 1741, 'kinda': 1742, 'hook': 1743, 'lone': 1744, 'foreign': 1745, 'safe': 1746, 'numer': 1747, 'rank': 1748, 'shape': 1749, 'washington': 1750, 'werewolf': 1751, 'rose': 1752, 'vs': 1753, 'civil': 1754, 'shallow': 1755, 'gari': 1756, 'morn': 1757, 'accomplish': 1758, 'kong': 1759, 'ordinari': 1760, 'winner': 1761, 'peac': 1762, 'grab': 1763, 'whenev': 1764, 'virtual': 1765, 'h': 1766, 'offens': 1767, 'luck': 1768, 'patient': 1769, 'welcom': 1770, 'unfunni': 1771, 'complain': 1772, 'bigger': 1773, 'activ': 1774, 'contriv': 1775, 'dimension': 1776, 'pretend': 1777, 'trek': 1778, 'con': 1779, 'cain': 1780, 'dri': 1781, 'lesbian': 1782, 'eric': 1783, 'code': 1784, 'flash': 1785, 'wake': 1786, 'manipul': 1787, 'corrupt': 1788, 'guard': 1789, 'albert': 1790, 'statu': 1791, 'dancer': 1792, 'signific': 1793, 'sourc': 1794, 'gain': 1795, 'awkward': 1796, 'context': 1797, 'speech': 1798, 'sean': 1799, 'psycho': 1800, '13': 1801, 'anthoni': 1802, 'corni': 1803, 'clip': 1804, 'reli': 1805, 'religion': 1806, 'w': 1807, 'theatric': 1808, 'curiou': 1809, 'advic': 1810, 'priest': 1811, 'flow': 1812, 'addict': 1813, 'specif': 1814, 'jennif': 1815, 'skin': 1816, 'asian': 1817, 'howard': 1818, 'secur': 1819, 'comfort': 1820, 'luke': 1821, 'promot': 1822, 'golden': 1823, 'core': 1824, 'organ': 1825, 'cash': 1826, 'cheat': 1827, 'lucki': 1828, 'associ': 1829, 'dislik': 1830, 'lower': 1831, 'regret': 1832, 'balanc': 1833, 'wing': 1834, 'devic': 1835, 'degre': 1836, 'spell': 1837, 'contribut': 1838, 'frankli': 1839, 'frequent': 1840, 'sake': 1841, 'lake': 1842, 'print': 1843, 'forgiv': 1844, 'thoma': 1845, 'mass': 1846, 'betti': 1847, 'crack': 1848, 'gordon': 1849, 'unexpect': 1850, 'unfold': 1851, 'construct': 1852, 'invit': 1853, 'grown': 1854, 'amateur': 1855, 'categori': 1856, 'depend': 1857, 'intellectu': 1858, 'matur': 1859, 'walter': 1860, 'grew': 1861, 'honor': 1862, 'condit': 1863, 'anna': 1864, 'veteran': 1865, 'sole': 1866, 'mirror': 1867, 'spectacular': 1868, 'sudden': 1869, 'meanwhil': 1870, 'experienc': 1871, 'grip': 1872, 'gift': 1873, 'robin': 1874, 'demonstr': 1875, 'overli': 1876, 'card': 1877, 'freedom': 1878, 'liner': 1879, 'crappi': 1880, 'brilliantli': 1881, 'theori': 1882, 'sheriff': 1883, 'subtitl': 1884, 'drew': 1885, 'circumst': 1886, 'section': 1887, 'unabl': 1888, 'oliv': 1889, 'colour': 1890, 'matt': 1891, 'parker': 1892, 'altern': 1893, 'sheer': 1894, 'path': 1895, 'pile': 1896, 'cook': 1897, 'laughter': 1898, 'hall': 1899, 'defin': 1900, 'accident': 1901, 'wander': 1902, 'sinatra': 1903, 'relief': 1904, 'treatment': 1905, 'lawyer': 1906, 'hank': 1907, 'captiv': 1908, 'dragon': 1909, 'halloween': 1910, 'gratuit': 1911, 'moor': 1912, 'wound': 1913, 'k': 1914, 'kung': 1915, 'jacki': 1916, 'cowboy': 1917, 'broadway': 1918, 'wayn': 1919, 'unintent': 1920, 'barbara': 1921, 'surreal': 1922, 'statement': 1923, 'canadian': 1924, 'winter': 1925, 'spoof': 1926, 'gonna': 1927, 'compos': 1928, 'treasur': 1929, 'fish': 1930, 'cheer': 1931, 'fare': 1932, 'victor': 1933, 'sensit': 1934, 'unrealist': 1935, 'emerg': 1936, 'woodi': 1937, 'driven': 1938, 'neighbor': 1939, 'ran': 1940, 'sympathet': 1941, 'overlook': 1942, 'glass': 1943, 'menac': 1944, 'authent': 1945, 'expos': 1946, 'topic': 1947, 'michel': 1948, 'handsom': 1949, 'chief': 1950, 'ancient': 1951, 'gross': 1952, 'russel': 1953, 'feet': 1954, 'network': 1955, 'stranger': 1956, 'comedian': 1957, 'pleasant': 1958, 'contemporari': 1959, 'nevertheless': 1960, 'built': 1961, 'cinderella': 1962, 'earn': 1963, 'blockbust': 1964, 'consider': 1965, 'endless': 1966, 'gori': 1967, 'letter': 1968, 'miser': 1969, 'underr': 1970, 'switch': 1971, 'brook': 1972, 'solv': 1973, 'joseph': 1974, 'virgin': 1975, 'edward': 1976, 'convict': 1977, 'victoria': 1978, 'bullet': 1979, 'chosen': 1980, '0': 1981, 'scale': 1982, 'cynic': 1983, 'scenario': 1984, 'alex': 1985, 'gut': 1986, 'sword': 1987, 'com': 1988, 'curs': 1989, 'outrag': 1990, 'juli': 1991, 'wrap': 1992, 'driver': 1993, 'proper': 1994, 'substanc': 1995, 'uk': 1996, 'monkey': 1997, 'screenwrit': 1998, 'court': 1999, 'par': 2000, 'bird': 2001, 'remov': 2002, 'indic': 2003, 'rental': 2004, 'advertis': 2005, 'nanci': 2006, 'roy': 2007, 'loser': 2008, 'consequ': 2009, 'inevit': 2010, 'naiv': 2011, 'grave': 2012, 'slap': 2013, 'brave': 2014, 'germani': 2015, 'le': 2016, 'bridg': 2017, 'invis': 2018, 'fatal': 2019, 'anger': 2020, 'provok': 2021, 'footbal': 2022, 'ador': 2023, 'loui': 2024, 'anderson': 2025, 'alcohol': 2026, 'chan': 2027, 'ryan': 2028, 'professor': 2029, 'stumbl': 2030, 'willi': 2031, 'assassin': 2032, 'sharp': 2033, 'australian': 2034, '1930': 2035, 'bat': 2036, 'patrick': 2037, 'amateurish': 2038, 'liber': 2039, 'strongli': 2040, 'ape': 2041, 'lousi': 2042, 'saturday': 2043, 'cell': 2044, 'eight': 2045, 'trilog': 2046, 'deni': 2047, 'heck': 2048, 'refresh': 2049, 'sin': 2050, 'resid': 2051, 'san': 2052, 'justifi': 2053, 'vagu': 2054, 'indi': 2055, 'defeat': 2056, 'creator': 2057, 'mini': 2058, 'reput': 2059, 'terrifi': 2060, 'sympathi': 2061, 'endur': 2062, 'prevent': 2063, 'expert': 2064, 'tediou': 2065, 'tabl': 2066, 'task': 2067, 'employ': 2068, 'rival': 2069, 'basebal': 2070, 'trial': 2071, 'imit': 2072, 'che': 2073, 'offend': 2074, 'fairi': 2075, 'beach': 2076, 'dig': 2077, 'weekend': 2078, 'pitch': 2079, 'max': 2080, 'complaint': 2081, 'europ': 2082, 'risk': 2083, 'format': 2084, 'murphi': 2085, 'purchas': 2086, 'harsh': 2087, 'hype': 2088, 'tini': 2089, 'titan': 2090, 'bite': 2091, 'glimps': 2092, 'nois': 2093, 'reminisc': 2094, 'powel': 2095, 'fals': 2096, 'strip': 2097, 'till': 2098, 'north': 2099, 'prime': 2100, '14': 2101, 'asleep': 2102, 'descript': 2103, 'texa': 2104, 'destruct': 2105, 'revel': 2106, 'africa': 2107, 'semi': 2108, 'spin': 2109, 'surfac': 2110, 'arrest': 2111, 'sitcom': 2112, 'uninterest': 2113, 'inner': 2114, 'excess': 2115, 'maintain': 2116, 'twin': 2117, 'argu': 2118, 'makeup': 2119, 'massiv': 2120, 'controversi': 2121, 'hitchcock': 2122, 'dinosaur': 2123, 'ideal': 2124, 'reject': 2125, 'stare': 2126, 'kim': 2127, 'ludicr': 2128, 'expens': 2129, 'insist': 2130, 'melodrama': 2131, 'host': 2132, 'forest': 2133, 'nail': 2134, 'ga': 2135, 'ala': 2136, 'erot': 2137, 'supernatur': 2138, 'atroci': 2139, 'press': 2140, 'columbo': 2141, 'subplot': 2142, 'notch': 2143, 'dude': 2144, 'cant': 2145, 'identifi': 2146, 'presum': 2147, 'plagu': 2148, 'crude': 2149, 'forgett': 2150, 'closer': 2151, 'character': 2152, 'guest': 2153, 'method': 2154, 'princess': 2155, 'lion': 2156, 'foster': 2157, 'beast': 2158, 'border': 2159, 'landscap': 2160, 'ear': 2161, 'damag': 2162, 'previous': 2163, 'birth': 2164, 'pacino': 2165, 'jungl': 2166, 'bound': 2167, 'aunt': 2168, 'urban': 2169, 'accus': 2170, 'storytel': 2171, 'nude': 2172, 'propaganda': 2173, 'emma': 2174, 'thirti': 2175, 'chose': 2176, 'jess': 2177, 'guid': 2178, 'doll': 2179, '25': 2180, 'whoever': 2181, 'mainstream': 2182, 'mate': 2183, 'pet': 2184, 'warrior': 2185, 'gritti': 2186, 'size': 2187, 'poster': 2188, 'deadli': 2189, 'upset': 2190, 'cooper': 2191, 'latest': 2192, 'exact': 2193, 'merit': 2194, 'friday': 2195, 'blend': 2196, '1990': 2197, 'contact': 2198, 'wilson': 2199, 'ton': 2200, 'warner': 2201, 'sun': 2202, 'contest': 2203, 'settl': 2204, 'popul': 2205, 'corps': 2206, 'rough': 2207, 'buff': 2208, 'citizen': 2209, 'environ': 2210, 'pitt': 2211, 'bu': 2212, 'mgm': 2213, 'select': 2214, 'alic': 2215, 'metal': 2216, 'rat': 2217, 'overcom': 2218, 'widow': 2219, 'guilti': 2220, 'ted': 2221, 'revolut': 2222, 'lift': 2223, 'link': 2224, 'particip': 2225, 'matrix': 2226, 'afternoon': 2227, 'accompani': 2228, 'exagger': 2229, '1960': 2230, 'johnson': 2231, 'prostitut': 2232, 'corpor': 2233, 'moron': 2234, 'corner': 2235, 'clair': 2236, 'multipl': 2237, 'holm': 2238, 'sincer': 2239, 'friendli': 2240, 'leagu': 2241, 'hood': 2242, 'doom': 2243, 'instal': 2244, 'defend': 2245, 'junk': 2246, 'lugosi': 2247, 'sunday': 2248, 'advis': 2249, 'blah': 2250, 'string': 2251, 'aka': 2252, 'campi': 2253, 'irish': 2254, 'examin': 2255, 'hip': 2256, 'grim': 2257, 'pro': 2258, 'rachel': 2259, 'confid': 2260, 'shake': 2261, 'varieti': 2262, 'tight': 2263, 'shut': 2264, 'icon': 2265, 'medic': 2266, 'denni': 2267, 'goal': 2268, 'jaw': 2269, 'sullivan': 2270, 'mexican': 2271, 'directli': 2272, 'attach': 2273, 'vietnam': 2274, 'breast': 2275, 'sarah': 2276, 'legendari': 2277, 'terrorist': 2278, 'sentenc': 2279, 'dean': 2280, 'truck': 2281, 'prior': 2282, 'duke': 2283, 'courag': 2284, 'bourn': 2285, 'proceed': 2286, 'un': 2287, 'hong': 2288, 'nose': 2289, 'yell': 2290, 'split': 2291, 'donald': 2292, 'behav': 2293, 'entri': 2294, 'concentr': 2295, 'lifetim': 2296, 'stolen': 2297, 'swim': 2298, 'borrow': 2299, 'unconvinc': 2300, 'buri': 2301, 'forth': 2302, 'confess': 2303, 'jerk': 2304, 'crush': 2305, 'gather': 2306, 'everywher': 2307, 'lip': 2308, 'spite': 2309, 'pan': 2310, 'deliveri': 2311, 'california': 2312, 'julia': 2313, 'turkey': 2314, 'offici': 2315, 'proud': 2316, 'reward': 2317, 'flight': 2318, 'quest': 2319, 'freeman': 2320, 'china': 2321, 'downright': 2322, 'hoffman': 2323, 'jon': 2324, 'inept': 2325, 'betray': 2326, 'notori': 2327, 'worthwhil': 2328, 'fabul': 2329, 'fade': 2330, 'jail': 2331, 'sir': 2332, 'encourag': 2333, 'lazi': 2334, 'sink': 2335, 'bell': 2336, 'relev': 2337, 'cousin': 2338, 'lisa': 2339, 'storm': 2340, 'teeth': 2341, 'survivor': 2342, 'branagh': 2343, 'retard': 2344, 'shower': 2345, 'imageri': 2346, 'bag': 2347, 'susan': 2348, 'quirki': 2349, 'facial': 2350, 'finger': 2351, 'mexico': 2352, 'toler': 2353, 'tremend': 2354, 'bride': 2355, 'alright': 2356, 'trade': 2357, 'summari': 2358, 'stab': 2359, 'shark': 2360, 'hugh': 2361, 'von': 2362, 'blown': 2363, 'bitter': 2364, 'hyster': 2365, 'pose': 2366, 'ha': 2367, 'christ': 2368, 'ned': 2369, 'address': 2370, 'bone': 2371, 'larri': 2372, 'scheme': 2373, 'afterward': 2374, 'cruel': 2375, 'ron': 2376, 'pursu': 2377, 'traci': 2378, 'thumb': 2379, 'snake': 2380, 'distinct': 2381, 'screw': 2382, 'feed': 2383, 'swear': 2384, 'beg': 2385, 'tour': 2386, 'chair': 2387, 'raw': 2388, 'photo': 2389, 'stomach': 2390, 'occas': 2391, 'mechan': 2392, 'obscur': 2393, 'cabin': 2394, 'southern': 2395, 'resist': 2396, 'chain': 2397, 'sidney': 2398, 'argument': 2399, 'necessarili': 2400, 'gruesom': 2401, 'holiday': 2402, 'heavili': 2403, 'render': 2404, 'hardi': 2405, 'indulg': 2406, 'philip': 2407, 'india': 2408, 'satan': 2409, 'racist': 2410, 'understood': 2411, 'tongu': 2412, 'belov': 2413, 'fourth': 2414, 'pregnant': 2415, 'midnight': 2416, 'integr': 2417, 'obnoxi': 2418, 'outfit': 2419, 'stalk': 2420, 'forgot': 2421, 'lay': 2422, 'restor': 2423, 'ticket': 2424, '17': 2425, 'garden': 2426, 'magazin': 2427, 'deeper': 2428, 'slapstick': 2429, 'inhabit': 2430, 'carol': 2431, 'brad': 2432, 'devot': 2433, 'shoe': 2434, 'incid': 2435, 'lincoln': 2436, 'benefit': 2437, 'anticip': 2438, 'underground': 2439, 'guarante': 2440, 'elizabeth': 2441, 'maria': 2442, 'lili': 2443, 'disbelief': 2444, 'divorc': 2445, 'sandler': 2446, 'explod': 2447, 'cring': 2448, 'bbc': 2449, 'princip': 2450, 'mildli': 2451, 'creation': 2452, 'amazingli': 2453, 'greater': 2454, 'capit': 2455, 'slave': 2456, 'introduct': 2457, 'extraordinari': 2458, 'funnier': 2459, 'lesli': 2460, 'halfway': 2461, 'extent': 2462, 'enhanc': 2463, 'text': 2464, 'punish': 2465, 'wreck': 2466, 'overwhelm': 2467, 'tap': 2468, 'advantag': 2469, 'transfer': 2470, 'east': 2471, 'horrif': 2472, 'lo': 2473, 'preview': 2474, 'lane': 2475, 'jessica': 2476, 'error': 2477, 'deliber': 2478, 'dynam': 2479, 'plant': 2480, 'appli': 2481, 'homosexu': 2482, 'vincent': 2483, 'basi': 2484, 'ensu': 2485, 'vacat': 2486, '2000': 2487, 'miller': 2488, 'miscast': 2489, 'sophist': 2490, 'mansion': 2491, 'elev': 2492, 'extend': 2493, 'spoken': 2494, 'sleazi': 2495, 'steel': 2496, 'bollywood': 2497, 'via': 2498, 'reed': 2499, 'uncomfort': 2500, 'measur': 2501, 'alter': 2502, 'fix': 2503, 'assign': 2504, 'hippi': 2505, 'mous': 2506, 'overact': 2507, 'savag': 2508, 'daili': 2509, 'breathtak': 2510, 'conceiv': 2511, 'beer': 2512, 'blair': 2513, 'goofi': 2514, 'dentist': 2515, 'melt': 2516, 'cathol': 2517, 'stanley': 2518, 'oppos': 2519, 'burt': 2520, 'succe': 2521, 'nowaday': 2522, 'subsequ': 2523, 'properli': 2524, 'inspector': 2525, 'carpent': 2526, 'everyday': 2527, 'sacrific': 2528, 'massacr': 2529, 'circl': 2530, 'block': 2531, 'neck': 2532, 'laura': 2533, 'access': 2534, 'pool': 2535, 'fallen': 2536, 'christi': 2537, 'mob': 2538, 'lesser': 2539, 'fay': 2540, 'portrait': 2541, 'grey': 2542, 'seagal': 2543, 'concert': 2544, 'relax': 2545, 'chees': 2546, 'usa': 2547, 'react': 2548, 'isol': 2549, 'sinist': 2550, 'competit': 2551, 'jake': 2552, 'jewish': 2553, 'spiritu': 2554, 'creep': 2555, 'stink': 2556, 'lyric': 2557, 'nine': 2558, '2006': 2559, 'suitabl': 2560, 'chop': 2561, 'ironi': 2562, 'appal': 2563, 'immens': 2564, 'nonetheless': 2565, 'rage': 2566, 'spring': 2567, 'sold': 2568, 'reduc': 2569, 'user': 2570, 'adopt': 2571, 'nut': 2572, 'franchis': 2573, 'showcas': 2574, 'retir': 2575, 'luci': 2576, 'navi': 2577, 'needless': 2578, 'shirt': 2579, 'digit': 2580, 'per': 2581, 'jay': 2582, 'uninspir': 2583, 'bulli': 2584, 'nurs': 2585, 'bath': 2586, 'asham': 2587, 'stanwyck': 2588, 'zone': 2589, '1940': 2590, '2001': 2591, 'laid': 2592, 'illustr': 2593, 'broadcast': 2594, 'amongst': 2595, 'sutherland': 2596, 'oddli': 2597, 'upper': 2598, 'fulfil': 2599, 'baker': 2600, 'throat': 2601, 'aspir': 2602, 'stylish': 2603, 'brando': 2604, 'disguis': 2605, 'nobl': 2606, 'em': 2607, 'endear': 2608, 'wanna': 2609, '18': 2610, 'pound': 2611, 'pride': 2612, 'impli': 2613, 'neighborhood': 2614, 'wwii': 2615, 'thief': 2616, 'dawn': 2617, 'diseas': 2618, '16': 2619, 'tens': 2620, 'shoulder': 2621, 'rochest': 2622, 'prop': 2623, 'coher': 2624, 'cinematograph': 2625, 'shift': 2626, 'dinner': 2627, 'albeit': 2628, 'distribut': 2629, 'bo': 2630, 'bett': 2631, 'poignant': 2632, 'shout': 2633, 'forti': 2634, 'silenc': 2635, 'contract': 2636, 'snow': 2637, 'rebel': 2638, 'wash': 2639, 'knife': 2640, 'function': 2641, 'surf': 2642, 'matthau': 2643, 'heat': 2644, 'horrend': 2645, 'mindless': 2646, 'cannib': 2647, 'elvira': 2648, 'henc': 2649, 'cancel': 2650, 'duti': 2651, 'internet': 2652, 'height': 2653, 'derek': 2654, 'silver': 2655, 'eeri': 2656, 'widmark': 2657, 'instinct': 2658, 'reunion': 2659, 'proof': 2660, 'chuck': 2661, 'incoher': 2662, 'repetit': 2663, 'absorb': 2664, 'alik': 2665, 'etern': 2666, 'neat': 2667, 'pie': 2668, 'premier': 2669, 'glori': 2670, 'greatli': 2671, 'innov': 2672, 'torn': 2673, 'mill': 2674, 'elvi': 2675, 'spielberg': 2676, 'musician': 2677, 'bang': 2678, 'crisi': 2679, 'itali': 2680, 'horrifi': 2681, 'blank': 2682, 'lovabl': 2683, 'burton': 2684, 'announc': 2685, 'redempt': 2686, 'homag': 2687, 'precis': 2688, 'infam': 2689, 'britain': 2690, 'trite': 2691, 'fbi': 2692, 'racism': 2693, 'wealthi': 2694, 'nelson': 2695, 'diamond': 2696, 'flop': 2697, 'hammer': 2698, 'resolut': 2699, 'parallel': 2700, 'chaplin': 2701, 'ensembl': 2702, 'dedic': 2703, 'happili': 2704, 'pat': 2705, 'streisand': 2706, 'helen': 2707, 'wilder': 2708, 'st': 2709, 'conclud': 2710, 'oil': 2711, 'mar': 2712, 'broke': 2713, 'carter': 2714, 'disagre': 2715, 'factori': 2716, 'triumph': 2717, 'plastic': 2718, 'cube': 2719, 'vega': 2720, 'climb': 2721, 'own': 2722, 'chuckl': 2723, 'row': 2724, 'rocket': 2725, 'weight': 2726, 'bush': 2727, 'march': 2728, 'fighter': 2729, 'kurt': 2730, 'dump': 2731, 'sensibl': 2732, 'dane': 2733, 'enorm': 2734, 'boot': 2735, 'thug': 2736, 'luca': 2737, 'unforgett': 2738, 'spare': 2739, 'wherea': 2740, 'meaning': 2741, 'lust': 2742, 'mst3k': 2743, 'engin': 2744, 'adequ': 2745, 'karloff': 2746, 'difficulti': 2747, 'threat': 2748, 'bobbi': 2749, 'dear': 2750, 'caricatur': 2751, 'stress': 2752, 'fifti': 2753, 'arnold': 2754, 'butt': 2755, 'brand': 2756, 'rap': 2757, 'arrog': 2758, 'polish': 2759, 'elabor': 2760, 'ego': 2761, 'homeless': 2762, 'hamlet': 2763, 'secretari': 2764, 'fest': 2765, 'swing': 2766, 'ralph': 2767, 'journalist': 2768, 'flynn': 2769, 'barri': 2770, 'induc': 2771, 'grate': 2772, 'fanci': 2773, 'spike': 2774, 'float': 2775, 'arrang': 2776, 'resort': 2777, 'tool': 2778, 'conspiraci': 2779, 'unbear': 2780, 'puppet': 2781, 'simpson': 2782, 'muppet': 2783, 'exercis': 2784, 'pig': 2785, 'cruis': 2786, 'guilt': 2787, 'boll': 2788, 'basement': 2789, 'phillip': 2790, 'tribut': 2791, 'choreograph': 2792, 'slip': 2793, 'ward': 2794, 'ham': 2795, 'tower': 2796, 'editor': 2797, 'file': 2798, 'babe': 2799, 'document': 2800, 'toilet': 2801, 'puzzl': 2802, '24': 2803, 'medium': 2804, 'stan': 2805, 'layer': 2806, 'korean': 2807, 'fianc': 2808, 'item': 2809, 'scarecrow': 2810, 'assur': 2811, 'spark': 2812, 'denzel': 2813, 'inexplic': 2814, 'philosoph': 2815, 'glover': 2816, 'catherin': 2817, 'slaughter': 2818, 'minim': 2819, 'superfici': 2820, 'larger': 2821, 'librari': 2822, 'transit': 2823, 'portion': 2824, 'persona': 2825, 'doc': 2826, 'orient': 2827, 'territori': 2828, 'boredom': 2829, 'pg': 2830, 'dorothi': 2831, 'owe': 2832, 'wolf': 2833, 'sneak': 2834, 'curti': 2835, 'ban': 2836, 'shi': 2837, 'walken': 2838, 'jeremi': 2839, 'financi': 2840, 'jet': 2841, 'profound': 2842, 'cusack': 2843, 'hudson': 2844, 'metaphor': 2845, 'eleph': 2846, 'backdrop': 2847, 'ambigu': 2848, 'multi': 2849, 'whale': 2850, '2005': 2851, 'notion': 2852, 'stiff': 2853, 'birthday': 2854, 'rave': 2855, 'union': 2856, 'viru': 2857, 'hack': 2858, 'implaus': 2859, 'elsewher': 2860, 'ultra': 2861, 'gadget': 2862, 'reader': 2863, 'superhero': 2864, 'squar': 2865, 'urg': 2866, 'slight': 2867, 'distanc': 2868, 'newspap': 2869, '1st': 2870, 'disc': 2871, 'poison': 2872, 'deriv': 2873, 'pad': 2874, 'canada': 2875, 'eastwood': 2876, 'afford': 2877, 'eva': 2878, 'hawk': 2879, 'bibl': 2880, 'lloyd': 2881, 'health': 2882, 'essenc': 2883, 'huh': 2884, 'skit': 2885, 'sadist': 2886, 'cure': 2887, 'restaur': 2888, 'drown': 2889, 'spread': 2890, 'charisma': 2891, 'montag': 2892, 'heston': 2893, 'button': 2894, 'invest': 2895, 'companion': 2896, 'peak': 2897, 'estat': 2898, 'lab': 2899, 'gradual': 2900, 'maniac': 2901, 'fetch': 2902, 'dealt': 2903, 'godfath': 2904, 'scoobi': 2905, 'muslim': 2906, 'alli': 2907, 'gothic': 2908, 'countless': 2909, 'ritter': 2910, 'cup': 2911, 'tea': 2912, 'servant': 2913, 'subtleti': 2914, 'kane': 2915, 'miik': 2916, 'electr': 2917, 'briefli': 2918, 'salli': 2919, 'heroic': 2920, 'elect': 2921, 'iii': 2922, 'charismat': 2923, 'nuanc': 2924, 'wannab': 2925, 'toss': 2926, 'bud': 2927, 'reel': 2928, 'ingredi': 2929, 'cole': 2930, 'resourc': 2931, 'tender': 2932, 'grandmoth': 2933, 'admittedli': 2934, 'neil': 2935, 'shall': 2936, 'mild': 2937, 'label': 2938, 'stood': 2939, 'gate': 2940, 'dawson': 2941, 'carrey': 2942, 'pauli': 2943, 'pit': 2944, 'poverti': 2945, 'punk': 2946, 'reev': 2947, 'mafia': 2948, 'stronger': 2949, 'kubrick': 2950, 'terri': 2951, 'cox': 2952, 'burst': 2953, 'outcom': 2954, 'tag': 2955, 'cardboard': 2956, 'easier': 2957, 'smash': 2958, 'assault': 2959, 'useless': 2960, 'smooth': 2961, 'ian': 2962, 'bakshi': 2963, 'fond': 2964, 'updat': 2965, 'astair': 2966, 'fist': 2967, 'coincid': 2968, 'sketch': 2969, 'vulner': 2970, 'melodramat': 2971, 'vari': 2972, '2002': 2973, 'rex': 2974, 'divers': 2975, 'resolv': 2976, 'samurai': 2977, 'increasingli': 2978, 'exchang': 2979, 'qualifi': 2980, 'reynold': 2981, 'scratch': 2982, 'tame': 2983, 'suspend': 2984, 'blast': 2985, 'luckili': 2986, 'be': 2987, 'conveni': 2988, 'insert': 2989, 'brillianc': 2990, 'templ': 2991, 'farm': 2992, 'matthew': 2993, 'fisher': 2994, 'seventi': 2995, 'coach': 2996, 'walker': 2997, 'hamilton': 2998, 'gotta': 2999, 'strictli': 3000, 'pin': 3001, 'ambiti': 3002, 'nuclear': 3003, 'jami': 3004, 'soprano': 3005, 'meat': 3006, 'timeless': 3007, 'brosnan': 3008, 'clock': 3009, 'eccentr': 3010, 'convolut': 3011, 'instantli': 3012, 'grasp': 3013, 'kudo': 3014, 'joey': 3015, 'ninja': 3016, 'spooki': 3017, 'revers': 3018, 'closet': 3019, 'cave': 3020, 'recreat': 3021, 'discoveri': 3022, 'butcher': 3023, 'worthless': 3024, 'empir': 3025, 'struck': 3026, 'monk': 3027, 'declar': 3028, 'importantli': 3029, 'bleak': 3030, 'seller': 3031, 'partli': 3032, 'miracl': 3033, 'cliff': 3034, 'mitchel': 3035, 'selfish': 3036, 'fifteen': 3037, 'wipe': 3038, 'pal': 3039, 'communist': 3040, 'clown': 3041, 'eighti': 3042, 'inconsist': 3043, 'gray': 3044, 'evok': 3045, 'sloppi': 3046, 'sidekick': 3047, 'norman': 3048, 'psychiatrist': 3049, 'flawless': 3050, 'chew': 3051, 'seed': 3052, 'destin': 3053, 'aforement': 3054, 'websit': 3055, '45': 3056, 'australia': 3057, 'ho': 3058, 'enthusiast': 3059, 'farc': 3060, 'superbl': 3061, 'cheek': 3062, 'debat': 3063, 'lifestyl': 3064, 'piano': 3065, 'stoog': 3066, 'splatter': 3067, 'wick': 3068, 'wrestl': 3069, 'dash': 3070, 'pressur': 3071, 'directori': 3072, 'bash': 3073, 'incompet': 3074, 'slice': 3075, 'drivel': 3076, 'emili': 3077, 'regardless': 3078, 'kitchen': 3079, 'dire': 3080, 'soviet': 3081, 'akshay': 3082, 'anni': 3083, 'abc': 3084, 'pleasantli': 3085, 'ken': 3086, 'increas': 3087, 'distant': 3088, 'duo': 3089, 'mann': 3090, 'beaten': 3091, 'prize': 3092, 'flower': 3093, 'dave': 3094, 'judi': 3095, 'glow': 3096, 'chapter': 3097, 'recov': 3098, 'curios': 3099, 'helicopt': 3100, 'suppli': 3101, 'cia': 3102, 'boil': 3103, 'cameron': 3104, 'seduc': 3105, 'cagney': 3106, 'artifici': 3107, 'jar': 3108, 'doo': 3109, 'lou': 3110, 'blob': 3111, 'combat': 3112, 'ellen': 3113, 'glenn': 3114, 'francisco': 3115, 'craig': 3116, 'craven': 3117, 'ranger': 3118, 'drunken': 3119, 'hop': 3120, 'web': 3121, 'favour': 3122, 'turner': 3123, 'splendid': 3124, 'psychot': 3125, 'laurel': 3126, 'perri': 3127, 'panic': 3128, 'goldberg': 3129, 'eleg': 3130, 'wizard': 3131, 'min': 3132, 'fx': 3133, 'shortli': 3134, 'alexand': 3135, 'hatr': 3136, 'slightest': 3137, 'graduat': 3138, 'rid': 3139, 'ruth': 3140, 'philosophi': 3141, '20th': 3142, 'plausibl': 3143, 'gentl': 3144, 'flip': 3145, 'falk': 3146, 'gandhi': 3147, 'modesti': 3148, 'greek': 3149, 'dracula': 3150, 'knight': 3151, 'ocean': 3152, 'jealou': 3153, 'manhattan': 3154, 'unpleas': 3155, 'legal': 3156, 'futurist': 3157, 'preciou': 3158, 'fund': 3159, 'felix': 3160, 'lend': 3161, 'holi': 3162, 'tall': 3163, 'harm': 3164, 'we': 3165, 'reviv': 3166, 'overdon': 3167, 'thread': 3168, 'digniti': 3169, 'nod': 3170, 'childish': 3171, 'mock': 3172, 'explicit': 3173, 'ami': 3174, 'scientif': 3175, 'bless': 3176, 'tank': 3177, 'forbidden': 3178, 'giallo': 3179, 'awe': 3180, 'nerv': 3181, 'yesterday': 3182, 'repeatedli': 3183, '99': 3184, 'broad': 3185, 'margaret': 3186, 'eve': 3187, 'elderli': 3188, 'unwatch': 3189, 'mel': 3190, 'verhoeven': 3191, 'torment': 3192, '2004': 3193, 'thick': 3194, 'pirat': 3195, 'awaken': 3196, 'fever': 3197, 'politician': 3198, 'rivet': 3199, 'absenc': 3200, 'publish': 3201, 'bin': 3202, 'timothi': 3203, 'custom': 3204, 'royal': 3205, 'automat': 3206, 'lean': 3207, 'acclaim': 3208, 'kay': 3209, 'uniform': 3210, 'ah': 3211, 'griffith': 3212, 'roman': 3213, 'romero': 3214, 'stiller': 3215, 'ambit': 3216, 'launch': 3217, 'eas': 3218, 'pierc': 3219, 'warren': 3220, 'sunshin': 3221, 'phrase': 3222, 'transport': 3223, 'homicid': 3224, 'antic': 3225, 'wallac': 3226, 'darker': 3227, 'pulp': 3228, 'foul': 3229, 'bathroom': 3230, 'termin': 3231, 'stinker': 3232, 'crook': 3233, 'tomato': 3234, 'purpl': 3235, 'gabriel': 3236, 'rambo': 3237, 'saint': 3238, 'sixti': 3239, 'album': 3240, 'evolv': 3241, 'donna': 3242, 'kenneth': 3243, 'revolutionari': 3244, 'eyr': 3245, 'marin': 3246, 'brazil': 3247, 'viciou': 3248, 'coloni': 3249, 'karen': 3250, 'contrari': 3251, '2003': 3252, 'packag': 3253, 'pray': 3254, 'ought': 3255, 'prom': 3256, 'awak': 3257, 'horrid': 3258, 'choreographi': 3259, 'hollow': 3260, 'juvenil': 3261, 'q': 3262, 'li': 3263, 'dose': 3264, 'beatti': 3265, 'option': 3266, 'stole': 3267, 'ireland': 3268, 'overr': 3269, 'ramon': 3270, 'conserv': 3271, 'candid': 3272, 'twelv': 3273, 'boast': 3274, 'nerd': 3275, 'defi': 3276, 'kapoor': 3277, 'blade': 3278, 'mummi': 3279, 'mildr': 3280, 'protest': 3281, 'collabor': 3282, 'fulci': 3283, 'astonish': 3284, 'funer': 3285, 'kirk': 3286, 'detract': 3287, 'confirm': 3288, 'trio': 3289, 'jazz': 3290, 'flame': 3291, 'global': 3292, 'natali': 3293, 'altman': 3294, 'destini': 3295, 'delici': 3296, 'bottl': 3297, 'mystic': 3298, 'tommi': 3299, 'enterpris': 3300, 'bull': 3301, 'spit': 3302, 'nicholson': 3303, 'blake': 3304, 'shade': 3305, 'racial': 3306, 'whip': 3307, 'yellow': 3308, 'audio': 3309, 'leap': 3310, 'todd': 3311, 'reunit': 3312, 'staff': 3313, 'meaningless': 3314, 'neo': 3315, 'inherit': 3316, 'vivid': 3317, 'merci': 3318, 'popcorn': 3319, 'threw': 3320, 'visibl': 3321, 'swedish': 3322, 'altogeth': 3323, 'pseudo': 3324, 'harder': 3325, 'enchant': 3326, 'bedroom': 3327, 'adolesc': 3328, 'fonda': 3329, 'ruthless': 3330, 'befriend': 3331, 'bust': 3332, 'madonna': 3333, 'moodi': 3334, 'uneven': 3335, 'tip': 3336, 'roommat': 3337, 'decor': 3338, 'kennedi': 3339, 'crocodil': 3340, 'await': 3341, 'leonard': 3342, 'edi': 3343, 'suspici': 3344, 'reserv': 3345, 'exhibit': 3346, 'voight': 3347, 'synopsi': 3348, 'respond': 3349, 'wire': 3350, 'lawrenc': 3351, 'lemmon': 3352, 'fanat': 3353, 'jew': 3354, 'atlanti': 3355, 'garner': 3356, 'clumsi': 3357, 'bargain': 3358, '2007': 3359, 'ventur': 3360, 'unsettl': 3361, 'rural': 3362, 'carl': 3363, 'clint': 3364, 'chao': 3365, 'abysm': 3366, 'dimens': 3367, 'centr': 3368, 'incident': 3369, 'voyag': 3370, 'audit': 3371, 'holli': 3372, 'palma': 3373, 'bradi': 3374, 'bold': 3375, 'lit': 3376, 'versu': 3377, 'elimin': 3378, 'daddi': 3379, 'mall': 3380, 'humili': 3381, 'wealth': 3382, 'hart': 3383, 'tiger': 3384, 'characterist': 3385, 'cari': 3386, 'trail': 3387, 'echo': 3388, 'ant': 3389, 'poetic': 3390, 'cd': 3391, 'imperson': 3392, '2nd': 3393, 'cuba': 3394, 'nearbi': 3395, 'acknowledg': 3396, 'neglect': 3397, 'troop': 3398, 'timon': 3399, 'immigr': 3400, 'solo': 3401, 'repuls': 3402, 'homer': 3403, 'collaps': 3404, 'paus': 3405, 'jeffrey': 3406, 'saga': 3407, 'celluloid': 3408, 'mistaken': 3409, 'infect': 3410, 'domest': 3411, 'pun': 3412, 'prejudic': 3413, 'mickey': 3414, 'marshal': 3415, 'cake': 3416, 'sore': 3417, 'inappropri': 3418, 'milk': 3419, 'interrupt': 3420, 'assembl': 3421, 'inan': 3422, 'coat': 3423, '1996': 3424, 'coffe': 3425, 'gear': 3426, 'chest': 3427, 'pant': 3428, 'tribe': 3429, 'hbo': 3430, 'undoubtedli': 3431, 'olivi': 3432, 'ginger': 3433, 'harvey': 3434, 'apolog': 3435, 'promin': 3436, 'equip': 3437, 'leon': 3438, 'colonel': 3439, 'aveng': 3440, 'primari': 3441, 'colleagu': 3442, 'exot': 3443, 'instant': 3444, 'airplan': 3445, 'solut': 3446, 'embrac': 3447, 'retain': 3448, 'furthermor': 3449, 'consum': 3450, 'trace': 3451, 'vulgar': 3452, 'pen': 3453, 'polanski': 3454, 'humbl': 3455, 'devast': 3456, 'institut': 3457, 'jenni': 3458, 'pot': 3459, 'maggi': 3460, 'brooklyn': 3461, 'highest': 3462, 'florida': 3463, 'ya': 3464, 'seduct': 3465, 'strain': 3466, 'sale': 3467, '1999': 3468, '3rd': 3469, 'dian': 3470, 'poke': 3471, 'outer': 3472, 'descend': 3473, 'rick': 3474, 'linda': 3475, 'dutch': 3476, 'bowl': 3477, 'gender': 3478, 'cope': 3479, 'godzilla': 3480, 'smaller': 3481, 'principl': 3482, 'wive': 3483, 'illog': 3484, 'disabl': 3485, 'dive': 3486, 'beneath': 3487, 'devoid': 3488, 'secondli': 3489, 'inferior': 3490, 'lol': 3491, 'scope': 3492, 'yard': 3493, 'gundam': 3494, 'hal': 3495, 'rabbit': 3496, 'dud': 3497, 'blatant': 3498, 'vast': 3499, 'predecessor': 3500, 'glamor': 3501, 'primarili': 3502, 'gloriou': 3503, 'cue': 3504, 'mixtur': 3505, 'bubbl': 3506, 'breed': 3507, 'museum': 3508, 'streep': 3509, 'et': 3510, 'arab': 3511, 'casual': 3512, 'grinch': 3513, 'alert': 3514, 'z': 3515, 'trademark': 3516, 'april': 3517, 'countrysid': 3518, 'domino': 3519, 'disjoint': 3520, 'hideou': 3521, 'invas': 3522, 'shelf': 3523, 'alfr': 3524, 'senseless': 3525, 'shirley': 3526, 'simplist': 3527, 'pearl': 3528, 'aggress': 3529, 'garbo': 3530, 'myer': 3531, 'talki': 3532, 'stir': 3533, 'mayor': 3534, 'unhappi': 3535, 'acid': 3536, 'stack': 3537, 'uwe': 3538, 'oz': 3539, 'sh': 3540, 'defens': 3541, 'experiment': 3542, 'khan': 3543, 'loyal': 3544, 'vanish': 3545, 'hopeless': 3546, 'obtain': 3547, 'stellar': 3548, 'mail': 3549, 'robinson': 3550, 'disgrac': 3551, 'rendit': 3552, 'boom': 3553, 'illeg': 3554, 'applaud': 3555, 'maci': 3556, 'slide': 3557, 'hardcor': 3558, 'robberi': 3559, 'wont': 3560, 'tempt': 3561, 'counter': 3562, 'dicken': 3563, 'topless': 3564, 'blew': 3565, 'scroog': 3566, 'rifl': 3567, 'fri': 3568, 'craze': 3569, 'tenant': 3570, 'grandfath': 3571, 'dismiss': 3572, 'span': 3573, 'incomprehens': 3574, 'recruit': 3575, 'hartley': 3576, 'spider': 3577, 'declin': 3578, 'emphasi': 3579, 'psychic': 3580, 'diana': 3581, 'berlin': 3582, 'soccer': 3583, 'amanda': 3584, 'bitch': 3585, 'niro': 3586, 'goer': 3587, 'porno': 3588, 'justin': 3589, 'parad': 3590, 'shed': 3591, 'resurrect': 3592, 'sympath': 3593, 'riot': 3594, 'ethnic': 3595, 'revolt': 3596, 'lumet': 3597, 'woo': 3598, 'wet': 3599, 'intim': 3600, 'ration': 3601, 'sibl': 3602, 'shaw': 3603, 'trashi': 3604, 'faster': 3605, 'patriot': 3606, 'eager': 3607, '00': 3608, 'enlighten': 3609, 'andr': 3610, 'commend': 3611, 'partial': 3612, 'feminist': 3613, 'mario': 3614, 'immort': 3615, 'nephew': 3616, 'honesti': 3617, 'jonathan': 3618, 'choru': 3619, 'slick': 3620, 'weakest': 3621, 'gap': 3622, 'region': 3623, 'worm': 3624, 'unreal': 3625, 'rider': 3626, 'hopper': 3627, 'ensur': 3628, 'steam': 3629, 'wheel': 3630, 'biographi': 3631, 'wendi': 3632, 'hesit': 3633, 'dealer': 3634, 'ballet': 3635, 'farmer': 3636, 'lena': 3637, 'repress': 3638, 'sandra': 3639, 'wore': 3640, 'mutant': 3641, 'safeti': 3642, 'morri': 3643, 'sappi': 3644, 'prequel': 3645, 'hung': 3646, 'psychopath': 3647, 'blunt': 3648, 'franco': 3649, 'vice': 3650, 'composit': 3651, 'nostalg': 3652, 'snap': 3653, 'confin': 3654, 'leo': 3655, 'owen': 3656, 'kingdom': 3657, 'charlott': 3658, 'macarthur': 3659, 'properti': 3660, 'victori': 3661, 'skull': 3662, 'similarli': 3663, 'util': 3664, 'snl': 3665, 'nervou': 3666, 'despair': 3667, 'exit': 3668, 'emperor': 3669, 'speci': 3670, 'repli': 3671, 'tad': 3672, 'bonu': 3673, 'strand': 3674, 'hyde': 3675, 'dalton': 3676, 'acquir': 3677, 'bumbl': 3678, 'dust': 3679, 'del': 3680, 'rocki': 3681, 'deed': 3682, 'recycl': 3683, 'compens': 3684, 'bow': 3685, 'drum': 3686, 'compass': 3687, 'farrel': 3688, 'drain': 3689, 'campbel': 3690, 'miseri': 3691, 'latin': 3692, 'thru': 3693, '1972': 3694, 'tail': 3695, 'montana': 3696, 'kyle': 3697, 'rope': 3698, 'bergman': 3699, 'cg': 3700, 'rambl': 3701, 'valuabl': 3702, 'whoopi': 3703, 'pattern': 3704, 'heartbreak': 3705, 'oppress': 3706, 'airport': 3707, 'radic': 3708, 'percept': 3709, 'rotten': 3710, 'roth': 3711, '35': 3712, 'rapist': 3713, 'tonight': 3714, 'carradin': 3715, 'gal': 3716, 'mistress': 3717, 'romp': 3718, 'slug': 3719, 'chess': 3720, 'bleed': 3721, 'downhil': 3722, 'martian': 3723, 'pour': 3724, 'wacki': 3725, 'gimmick': 3726, 'contempl': 3727, 'olli': 3728, 'orson': 3729, 'da': 3730, 'pursuit': 3731, 'edgar': 3732, 'banal': 3733, 'stilt': 3734, 'arc': 3735, 'belt': 3736, 'programm': 3737, 'heal': 3738, 'pervert': 3739, 'shelley': 3740, 'dazzl': 3741, 'slash': 3742, 'arguabl': 3743, 'tackl': 3744, 'tooth': 3745, 'mislead': 3746, 'unpredict': 3747, '1983': 3748, 'champion': 3749, 'paltrow': 3750, 'pervers': 3751, 'preach': 3752, 'attorney': 3753, 'melodi': 3754, 'taught': 3755, 'employe': 3756, 'bela': 3757, 'sensat': 3758, 'uplift': 3759, 'rubi': 3760, 'maid': 3761, 'vocal': 3762, 'chicken': 3763, 'orang': 3764, 'mesmer': 3765, 'gambl': 3766, 'duval': 3767, 'franki': 3768, 'passeng': 3769, 'graham': 3770, 'tiresom': 3771, 'cleverli': 3772, 'raymond': 3773, 'plight': 3774, 'virginia': 3775, 'conneri': 3776, 'marti': 3777, 'dixon': 3778, 'closest': 3779, 'poem': 3780, 'vengeanc': 3781, 'mute': 3782, 'outing': 3783, 'climact': 3784, 'quarter': 3785, 'monologu': 3786, 'yawn': 3787, 'whine': 3788, 'clone': 3789, 'inject': 3790, 'tube': 3791, 'scottish': 3792, 'gerard': 3793, 'suffic': 3794, 'numb': 3795, 'amitabh': 3796, 'bay': 3797, 'convincingli': 3798, 'engross': 3799, 'extens': 3800, 'volum': 3801, 'profan': 3802, 'swallow': 3803, 'habit': 3804, 'giggl': 3805, 'paranoia': 3806, 'secretli': 3807, 'calm': 3808, 'pokemon': 3809, 'abraham': 3810, 'lundgren': 3811, 'sirk': 3812, 'crystal': 3813, '1968': 3814, 'iran': 3815, 'junior': 3816, 'franci': 3817, 'ethan': 3818, 'taxi': 3819, 'im': 3820, 'plod': 3821, 'backward': 3822, 'lowest': 3823, 'spock': 3824, 'poetri': 3825, 'linger': 3826, 'bend': 3827, 'frankenstein': 3828, 'nichola': 3829, 'underst': 3830, 'abort': 3831, 'grotesqu': 3832, 'profess': 3833, 'chicago': 3834, 'richardson': 3835, 'septemb': 3836, 'fed': 3837, 'austen': 3838, 'expand': 3839, 'earl': 3840, 'dispos': 3841, 'surpass': 3842, 'meander': 3843, 'trend': 3844, 'muddl': 3845, 'eugen': 3846, 'lure': 3847, 'rant': 3848, 'mundan': 3849, 'econom': 3850, 'household': 3851, 'waitress': 3852, 'dysfunct': 3853, 'cannon': 3854, 'hum': 3855, 'catchi': 3856, 'rubber': 3857, 'sue': 3858, 'compliment': 3859, 'greedi': 3860, 'der': 3861, 'myth': 3862, 'instrument': 3863, 'simplic': 3864, 'nostalgia': 3865, 'stallon': 3866, 'spoke': 3867, 'literatur': 3868, 'tourist': 3869, 'descent': 3870, 'crucial': 3871, 'phoni': 3872, 'randi': 3873, 'lang': 3874, 'molli': 3875, 'louis': 3876, 'occupi': 3877, 'stale': 3878, 'deaf': 3879, 'map': 3880, 'eaten': 3881, 'alongsid': 3882, 'coast': 3883, 'duck': 3884, 'phantom': 3885, 'recognit': 3886, 'irrelev': 3887, 'june': 3888, 'damon': 3889, 'mankind': 3890, 'hello': 3891, 'flee': 3892, 'mortal': 3893, 'recognis': 3894, 'equival': 3895, 'sissi': 3896, 'cent': 3897, 'furi': 3898, 'dement': 3899, 'bacal': 3900, 'carel': 3901, 'insur': 3902, 'firstli': 3903, 'dictat': 3904, 'omen': 3905, 'cyborg': 3906, 'damm': 3907, 'heel': 3908, 'antwon': 3909, 'bump': 3910, 'likewis': 3911, 'drake': 3912, 'ashley': 3913, 'wisdom': 3914, 'twilight': 3915, 'onlin': 3916, 'lengthi': 3917, 'loyalti': 3918, 'newli': 3919, '1973': 3920, 'freez': 3921, 'bike': 3922, 'daisi': 3923, 'distinguish': 3924, 'dreari': 3925, 'grayson': 3926, 'rude': 3927, 'labor': 3928, 'biko': 3929, 'blackmail': 3930, 'reign': 3931, 'buffalo': 3932, 'rooney': 3933, 'unorigin': 3934, 'provoc': 3935, 'tunnel': 3936, 'analysi': 3937, 'inher': 3938, 'pink': 3939, 'interior': 3940, 'incorpor': 3941, 'keith': 3942, 'approv': 3943, 'butler': 3944, 'worn': 3945, 'emphas': 3946, 'nineti': 3947, 'barrymor': 3948, 'attribut': 3949, 'exposur': 3950, 'vein': 3951, 'chronicl': 3952, 'ridden': 3953, 'startl': 3954, 'sailor': 3955, 'proce': 3956, 'boxer': 3957, 'basketbal': 3958, 'baddi': 3959, 'prey': 3960, 'elm': 3961, 'othello': 3962, 'barrel': 3963, 'meg': 3964, 'walsh': 3965, 'mormon': 3966, 'millionair': 3967, 'drift': 3968, 'improvis': 3969, 'julian': 3970, 'mighti': 3971, 'simmon': 3972, 'predat': 3973, 'fleet': 3974, 'condemn': 3975, 'robbin': 3976, 'hypnot': 3977, 'indiffer': 3978, 'unrel': 3979, 'nicol': 3980, 'meyer': 3981, 'substitut': 3982, 'carla': 3983, 'stalker': 3984, 'belushi': 3985, 'undeni': 3986, 'er': 3987, 'degrad': 3988, 'bunni': 3989, 'underli': 3990, 'rukh': 3991, 'shove': 3992, 'roof': 3993, 'vital': 3994, 'greed': 3995, 'disord': 3996, 'marion': 3997, 'alarm': 3998, 'exquisit': 3999, 'enthusiasm': 4000, 'edgi': 4001, 'priceless': 4002, 'reid': 4003, 'lampoon': 4004, 'alison': 4005, 'hay': 4006, 'warmth': 4007, 'palac': 4008, 'firm': 4009, 'watson': 4010, 'novak': 4011, 'unawar': 4012, 'dolph': 4013, '3d': 4014, 'errol': 4015, 'mtv': 4016, 'agenda': 4017, 'nyc': 4018, 'sergeant': 4019, 'distort': 4020, '1933': 4021, 'spain': 4022, 'glanc': 4023, 'peck': 4024, 'what': 4025, 'session': 4026, 'testament': 4027, 'crown': 4028, 'nun': 4029, 'unleash': 4030, 'angela': 4031, 'drip': 4032, 'preserv': 4033, 'minimum': 4034, 'ponder': 4035, 'simultan': 4036, 'profit': 4037, 'showdown': 4038, 'eastern': 4039, 'randomli': 4040, 'peril': 4041, 'thompson': 4042, 'coup': 4043, 'gestur': 4044, 'pamela': 4045, '13th': 4046, 'championship': 4047, 'beatl': 4048, 'cassidi': 4049, 'zizek': 4050, 'iraq': 4051, 'campaign': 4052, 'valentin': 4053, 'orlean': 4054, 'israel': 4055, 'petti': 4056, '1984': 4057, 'brenda': 4058, 'perpetu': 4059, 'reson': 4060, 'contradict': 4061, 'unimagin': 4062, 'crow': 4063, 'shootout': 4064, 'valley': 4065, 'rout': 4066, 'buster': 4067, 'exposit': 4068, 'stake': 4069, 'scotland': 4070, 'wig': 4071, 'kurosawa': 4072, 'cream': 4073, 'calib': 4074, 'realm': 4075, 'empathi': 4076, 'represent': 4077, 'stroke': 4078, 'din': 4079, 'travesti': 4080, 'cooki': 4081, 'gentleman': 4082, 'bro': 4083, 'climat': 4084, 'crawl': 4085, 'sabrina': 4086, 'mon': 4087, 'miyazaki': 4088, 'regist': 4089, 'quinn': 4090, 'han': 4091, 'restrain': 4092, 'fido': 4093, 'jan': 4094, 'warrant': 4095, 'greg': 4096, 'businessman': 4097, 'wax': 4098, 'meryl': 4099, 'distress': 4100, 'censor': 4101, 'tacki': 4102, 'perceiv': 4103, 'compromis': 4104, 'darren': 4105, 'painter': 4106, 'sammi': 4107, 'abomin': 4108, 'traumat': 4109, 'shaki': 4110, 'delic': 4111, 'baldwin': 4112, 'josh': 4113, '1997': 4114, 'unseen': 4115, 'soderbergh': 4116, 'monoton': 4117, 'pretens': 4118, 'passabl': 4119, 'ross': 4120, 'fuller': 4121, 'mclaglen': 4122, 'absent': 4123, 'femm': 4124, 'crawford': 4125, 'shoddi': 4126, 'pole': 4127, 'geek': 4128, '1987': 4129, 'demis': 4130, 'dana': 4131, 'spacey': 4132, 'stargat': 4133, 'unsatisfi': 4134, 'derang': 4135, 'sucker': 4136, 'cloud': 4137, 'ustinov': 4138, 'nathan': 4139, 'kumar': 4140, 'jewel': 4141, 'norm': 4142, 'valid': 4143, '1993': 4144, 'click': 4145, 'clash': 4146, 'judgment': 4147, 'unravel': 4148, 'expedit': 4149, 'uncov': 4150, 'austin': 4151, 'deceas': 4152, 'reluct': 4153, 'verbal': 4154, 'seal': 4155, 'anchor': 4156, 'exclus': 4157, 'antonioni': 4158, 'deniro': 4159, 'tarantino': 4160, 'furiou': 4161, 'accuraci': 4162, 'polici': 4163, 'correctli': 4164, 'primit': 4165, 'fenc': 4166, 'dee': 4167, 'sid': 4168, 'wholli': 4169, 'tech': 4170, 'fog': 4171, 'joel': 4172, '2008': 4173, 'conduct': 4174, 'behold': 4175, 'clerk': 4176, 'crippl': 4177, 'murray': 4178, 'mode': 4179, 'temper': 4180, 'hallucin': 4181, 'enforc': 4182, '3000': 4183, 'nicola': 4184, 'sand': 4185, 'fabric': 4186, '1971': 4187, 'dreck': 4188, 'slam': 4189, 'sunni': 4190, 'wretch': 4191, 'fought': 4192, 'logan': 4193, 'pocket': 4194, 'roller': 4195, 'sustain': 4196, 'unfair': 4197, 'trait': 4198, 'fart': 4199, 'bake': 4200, 'ritual': 4201, '1995': 4202, 'wang': 4203, 'debt': 4204, 'malon': 4205, 'tax': 4206, 'shanghai': 4207, 'patienc': 4208, 'alec': 4209, 'seldom': 4210, 'sheet': 4211, 'darn': 4212, 'vanc': 4213, 'runner': 4214, 'grief': 4215, 'scriptwrit': 4216, 'preposter': 4217, 'squad': 4218, 'bias': 4219, 'soup': 4220, 'stark': 4221, 'guitar': 4222, 'conscious': 4223, 'technicolor': 4224, 'despis': 4225, 'divid': 4226, 'tactic': 4227, 'helpless': 4228, 'pete': 4229, 'rita': 4230, 'stuart': 4231, 'sweep': 4232, 'fundament': 4233, 'robber': 4234, 'canyon': 4235, 'schedul': 4236, 'phil': 4237, 'bridget': 4238, 'penni': 4239, 'clau': 4240, 'exhaust': 4241, 'shell': 4242, 'isabel': 4243, 'critiqu': 4244, 'outlin': 4245, 'legaci': 4246, 'preston': 4247, 'passag': 4248, 'bloom': 4249, 'unexpectedli': 4250, 'lacklust': 4251, 'kansa': 4252, 'liberti': 4253, 'flair': 4254, 'restrict': 4255, 'rear': 4256, 'sentinel': 4257, 'alley': 4258, 'sniper': 4259, 'vomit': 4260, 'cigarett': 4261, 'marc': 4262, 'invad': 4263, 'jacket': 4264, 'rehash': 4265, 'downey': 4266, 'culmin': 4267, 'connor': 4268, 'russia': 4269, 'inabl': 4270, 'palanc': 4271, 'delv': 4272, 'gregori': 4273, 'boyl': 4274, 'agenc': 4275, 'drove': 4276, 'consciou': 4277, 'jodi': 4278, 'newman': 4279, 'alicia': 4280, 'sugar': 4281, 'propos': 4282, 'implic': 4283, 'asylum': 4284, 'horn': 4285, 'tripe': 4286, 'cap': 4287, 'lush': 4288, 'aesthet': 4289, '1936': 4290, 'pale': 4291, 'wrench': 4292, 'karl': 4293, 'rod': 4294, 'delet': 4295, 'behaviour': 4296, 'yeti': 4297, 'rampag': 4298, 'awhil': 4299, 'mccoy': 4300, 'rehears': 4301, '22': 4302, 'feat': 4303, 'foxx': 4304, 'vet': 4305, 'ladder': 4306, 'improb': 4307, 'chainsaw': 4308, 'arrow': 4309, 'tendenc': 4310, 'bacon': 4311, 'sharon': 4312, 'kolchak': 4313, 'tasteless': 4314, 'sung': 4315, 'el': 4316, 'suffici': 4317, 'hackney': 4318, 'wildli': 4319, 'rumor': 4320, 'newcom': 4321, 'elit': 4322, 'conscienc': 4323, 'financ': 4324, 'filler': 4325, 'lurk': 4326, 'weav': 4327, '19th': 4328, 'tomorrow': 4329, 'globe': 4330, 'prank': 4331, 'hulk': 4332, 'basing': 4333, 'underneath': 4334, '1988': 4335, 'fright': 4336, '1978': 4337, 'minu': 4338, 'spice': 4339, 'rhythm': 4340, 'scoop': 4341, 'coaster': 4342, 'amazon': 4343, 'visitor': 4344, 'loneli': 4345, '1920': 4346, 'paradis': 4347, 'suspicion': 4348, 'aristocrat': 4349, 'stream': 4350, 'hungri': 4351, 'shortcom': 4352, 'paramount': 4353, 'wagner': 4354, 'thunderbird': 4355, 'penn': 4356, 'cancer': 4357, 'counterpart': 4358, 'standout': 4359, 'chamberlain': 4360, 'worship': 4361, 'curli': 4362, 'entranc': 4363, 'paxton': 4364, 'literari': 4365, 'smell': 4366, 'brit': 4367, 'iv': 4368, 'en': 4369, 'grudg': 4370, 'atroc': 4371, 'lectur': 4372, 'couch': 4373, 'choppi': 4374, 'straightforward': 4375, '75': 4376, 'impos': 4377, 'rub': 4378, 'teas': 4379, 'recogniz': 4380, 'secondari': 4381, 'abrupt': 4382, 'chavez': 4383, 'bread': 4384, '1939': 4385, 'posey': 4386, 'ram': 4387, 'dirt': 4388, 'minist': 4389, 'hopkin': 4390, 'leigh': 4391, 'ingeni': 4392, 'inmat': 4393, 'immers': 4394, 'wwe': 4395, 'springer': 4396, '1989': 4397, 'heist': 4398, 'tierney': 4399, 'beverli': 4400, 'naughti': 4401, 'quietli': 4402, 'variat': 4403, 'moreov': 4404, 'yearn': 4405, 'net': 4406, 'quaid': 4407, 'heartfelt': 4408, 'bernard': 4409, 'transcend': 4410, 'convert': 4411, 'sublim': 4412, 'duel': 4413, 'ace': 4414, '1986': 4415, 'esther': 4416, 'laurenc': 4417, 'sassi': 4418, 'skeptic': 4419, 'injuri': 4420, 'enthral': 4421, 'nemesi': 4422, 'clan': 4423, 'misguid': 4424, 'policeman': 4425, 'missil': 4426, 'morbid': 4427, 'geni': 4428, 'entitl': 4429, 'cattl': 4430, 'nolan': 4431, 'lindsay': 4432, 'watcher': 4433, 'attenborough': 4434, 'ratso': 4435, 'graini': 4436, 'reliabl': 4437, 'vader': 4438, 'grin': 4439, 'uncut': 4440, 'tyler': 4441, 'cruelti': 4442, 'steadi': 4443, 'moder': 4444, 'kitti': 4445, 'puppi': 4446, 'spiral': 4447, 'diari': 4448, 'out': 4449, 'brood': 4450, 'hopelessli': 4451, 'bean': 4452, 'youngest': 4453, 'buzz': 4454, 'obstacl': 4455, 'kidman': 4456, 'bye': 4457, 'facil': 4458, 'artsi': 4459, 'poe': 4460, 'rosemari': 4461, 'egg': 4462, 'enabl': 4463, 'mytholog': 4464, '1979': 4465, 'dont': 4466, 'setup': 4467, 'characteris': 4468, 'unexplain': 4469, 'hk': 4470, 'carlito': 4471, 'christin': 4472, 'kline': 4473, 'effici': 4474, 'clueless': 4475, 'hain': 4476, 'patricia': 4477, 'bewar': 4478, 'athlet': 4479, 'oblig': 4480, 'despic': 4481, 'spontan': 4482, 'underworld': 4483, 'sweat': 4484, 'bounc': 4485, 'acquaint': 4486, 'hammi': 4487, 'exterior': 4488, 'heap': 4489, 'baffl': 4490, 'disastr': 4491, 'bronson': 4492, 'martha': 4493, 'weather': 4494, '1969': 4495, 'fuel': 4496, 'niec': 4497, 'gina': 4498, 'preming': 4499, 'narrow': 4500, 'gillian': 4501, 'decept': 4502, 'brendan': 4503, 'loath': 4504, 'trigger': 4505, 'outlaw': 4506, 'dandi': 4507, 'preachi': 4508, 'circu': 4509, 'shatter': 4510, 'enlist': 4511, 'rome': 4512, 'candl': 4513, 'scar': 4514, '73': 4515, 'harmless': 4516, 'tick': 4517, 'insipid': 4518, 'loi': 4519, 'sooner': 4520, 'housewif': 4521, 'headach': 4522, 'uh': 4523, 'analyz': 4524, 'astound': 4525, 'angst': 4526, 'suprem': 4527, '19': 4528, 'taboo': 4529, 'viewpoint': 4530, 'virtu': 4531, 'goof': 4532, 'sleepwalk': 4533, 'dilemma': 4534, 'hepburn': 4535, 'lester': 4536, 'biker': 4537, 'mermaid': 4538, 'fontain': 4539, 'injur': 4540, 'renaiss': 4541, 'mayhem': 4542, 'slimi': 4543, 'dismal': 4544, 'intric': 4545, 'hokey': 4546, 'salt': 4547, 'tripl': 4548, 'steer': 4549, 'glorifi': 4550, 'stimul': 4551, 'contempt': 4552, 'camcord': 4553, 'whore': 4554, 'hostag': 4555, 'amor': 4556, 'zoom': 4557, 'macho': 4558, 'hooker': 4559, 'claustrophob': 4560, 'foolish': 4561, 'fluff': 4562, 'stair': 4563, 'filth': 4564, 'idol': 4565, 'dish': 4566, 'boston': 4567, 'overlong': 4568, 'bent': 4569, 'ebert': 4570, 'corbett': 4571, 'cassavet': 4572, 'gere': 4573, 'phenomenon': 4574, 'spade': 4575, 'redund': 4576, 'oldest': 4577, 'scorses': 4578, 'sox': 4579, 'surgeri': 4580, 'guin': 4581, 'immatur': 4582, 'ariel': 4583, 'keen': 4584, 'cush': 4585, 'obligatori': 4586, 'perman': 4587, 'cohen': 4588, 'faint': 4589, 'muscl': 4590, 'widescreen': 4591, 'corman': 4592, 'frantic': 4593, 'remad': 4594, 'naschi': 4595, 'flirt': 4596, 'trivia': 4597, 'zane': 4598, 'proport': 4599, 'dwarf': 4600, 'nolt': 4601, 'flag': 4602, 'transplant': 4603, 'flashi': 4604, 'rhyme': 4605, 'mount': 4606, 'margin': 4607, 'conquer': 4608, 'cow': 4609, 'harold': 4610, 'mutual': 4611, 'astronaut': 4612, 'schlock': 4613, 'shred': 4614, 'spree': 4615, 'joker': 4616, 'antagonist': 4617, 'strongest': 4618, 'radiat': 4619, 'fascist': 4620, 'preced': 4621, 'messi': 4622, 'beard': 4623, 'down': 4624, 'gabl': 4625, 'assert': 4626, 'shield': 4627, 'alvin': 4628, '1981': 4629, 'spinal': 4630, 'gasp': 4631, '1976': 4632, 'bachelor': 4633, 'claud': 4634, 'sensual': 4635, 'raj': 4636, 'scandal': 4637, 'discern': 4638, 'bitten': 4639, 'archiv': 4640, 'divin': 4641, '95': 4642, 'wield': 4643, 'brush': 4644, 'repris': 4645, 'deer': 4646, 'mobil': 4647, 'interestingli': 4648, 'hara': 4649, 'departur': 4650, 'barn': 4651, 'boob': 4652, 'instruct': 4653, 'inflict': 4654, 'info': 4655, 'someday': 4656, '28': 4657, 'ritchi': 4658, 'www': 4659, 'triangl': 4660, 'mol': 4661, 'persuad': 4662, 'neurot': 4663, 'off': 4664, 'flock': 4665, 'strive': 4666, 'resum': 4667, 'aborigin': 4668, 'danish': 4669, 'carey': 4670, 'vaniti': 4671, 'fishburn': 4672, '1945': 4673, 'prophet': 4674, 'dylan': 4675, 'miracul': 4676, 'colin': 4677, 'neill': 4678, 'axe': 4679, 'vibrant': 4680, 'parson': 4681, 'hug': 4682, 'recit': 4683, 'dim': 4684, 'mobster': 4685, 'harrison': 4686, 'rot': 4687, 'traffic': 4688, 'fragil': 4689, 'hapless': 4690, 'luka': 4691, 'clad': 4692, 'proclaim': 4693, 'wendigo': 4694, 'melissa': 4695, 'pacif': 4696, 'submit': 4697, 'helm': 4698, 'casino': 4699, 'heartwarm': 4700, 'dame': 4701, 'hilar': 4702, 'ish': 4703, 'frontier': 4704, 'undermin': 4705, 'banter': 4706, 'cycl': 4707, 'carlo': 4708, 'earnest': 4709, 'senior': 4710, 'pixar': 4711, 'kathryn': 4712, 'cliffhang': 4713, 'cher': 4714, 'europa': 4715, 'bate': 4716, 'pickford': 4717, 'timberlak': 4718, 'artwork': 4719, 'anton': 4720, 'jade': 4721, 'cb': 4722, 'biblic': 4723, 'loretta': 4724, 'breakfast': 4725, 'antholog': 4726, 'bondag': 4727, 'seedi': 4728, 'holocaust': 4729, 'razor': 4730, 'vile': 4731, 'nope': 4732, 'eli': 4733, 'sicken': 4734, 'foil': 4735, 'redneck': 4736, 'cerebr': 4737, 'winchest': 4738, 'bikini': 4739, 'rooki': 4740, 'marlon': 4741, 'feast': 4742, 'aris': 4743, 'akin': 4744, 'legitim': 4745, 'articl': 4746, 'misfortun': 4747, 'northern': 4748, 'token': 4749, 'orphan': 4750, 'venom': 4751, 'http': 4752, 'uma': 4753, 'trier': 4754, 'lucil': 4755, 'flavor': 4756, 'illus': 4757, 'static': 4758, 'estrang': 4759, 'vanessa': 4760, 'choke': 4761, 'mason': 4762, 'pc': 4763, 'alexandr': 4764, 'jo': 4765, 'wardrob': 4766, 'toronto': 4767, 'electron': 4768, 'blatantli': 4769, 'jordan': 4770, 'lui': 4771, 'shepherd': 4772, 'isra': 4773, 'mathieu': 4774, 'milo': 4775, 'highway': 4776, 'deem': 4777, 'outdat': 4778, 'psych': 4779, 'glare': 4780, 'feminin': 4781, 'smack': 4782, 'ideolog': 4783, 'gilbert': 4784, 'clinic': 4785, 'fifth': 4786, 'ceremoni': 4787, 'shorter': 4788, 'affleck': 4789, 'cartoonish': 4790, 'styliz': 4791, 'gunga': 4792, 'linear': 4793, 'swept': 4794, 'frog': 4795, 'retriev': 4796, 'charlton': 4797, 'tack': 4798, 'dudley': 4799, 'abund': 4800, 'boyer': 4801, 'comprehend': 4802, 'knightley': 4803, 'wrestler': 4804, 'leather': 4805, 'magician': 4806, 'nightclub': 4807, 'howl': 4808, 'turd': 4809, 'peer': 4810, 'huston': 4811, 'disregard': 4812, 'oppon': 4813, 'audrey': 4814, 'spawn': 4815, 'summar': 4816, 'lifeless': 4817, 'toe': 4818, 'chip': 4819, 'btw': 4820, 'corn': 4821, 'braveheart': 4822, 'deliver': 4823, '1994': 4824, 'boo': 4825, 'breakdown': 4826, 'conrad': 4827, 'snatch': 4828, 'moe': 4829, 'bogu': 4830, 'sleaz': 4831, 'collector': 4832, '4th': 4833, 'plate': 4834, 'tara': 4835, 'lavish': 4836, 'compris': 4837, 'greet': 4838, 'newer': 4839, 'einstein': 4840, 'uniformli': 4841, 'cemeteri': 4842, 'cuban': 4843, 'senat': 4844, 'lighter': 4845, 'durat': 4846, 'phenomen': 4847, 'bastard': 4848, 'whack': 4849, 'mitch': 4850, 'spine': 4851, 'goldsworthi': 4852, 'potter': 4853, 'monument': 4854, 'energet': 4855, 'evolut': 4856, '1991': 4857, 'client': 4858, 'salman': 4859, 'jare': 4860, 'gilliam': 4861, 'pronounc': 4862, 'jedi': 4863, 'bori': 4864, 'firmli': 4865, 'fluid': 4866, 'appl': 4867, 'constitut': 4868, 'jam': 4869, 'clara': 4870, 'bulk': 4871, 'capot': 4872, 'jule': 4873, 'kent': 4874, 'embark': 4875, 'luxuri': 4876, 'outright': 4877, 'belli': 4878, 'trauma': 4879, 'evelyn': 4880, 'alleg': 4881, 'spectacl': 4882, 'neatli': 4883, 'undertak': 4884, '1974': 4885, 'ie': 4886, 'cecil': 4887, 'sorrow': 4888, 'ol': 4889, 'occup': 4890, 'replay': 4891, 'healthi': 4892, 'mcqueen': 4893, 'signal': 4894, 'historian': 4895, 'armstrong': 4896, 'inaccuraci': 4897, 'kazan': 4898, 'lex': 4899, 'nina': 4900, 'randolph': 4901, 'eleven': 4902, 'judd': 4903, 'wtf': 4904, 'creek': 4905, 'undead': 4906, '1977': 4907, 'liu': 4908, 'comprehens': 4909, 'genet': 4910, 'roar': 4911, 'vignett': 4912, 'cape': 4913, 'knee': 4914, 'conan': 4915, 'antonio': 4916, 'id': 4917, 'lanc': 4918, 'paula': 4919, 'mum': 4920, 'pepper': 4921, 'unsuspect': 4922, 'pioneer': 4923, 'congratul': 4924, '1985': 4925, 'tokyo': 4926, 'rosario': 4927, 'unattract': 4928, 'bait': 4929, 'bsg': 4930, 'truman': 4931, 'inclus': 4932, 'vain': 4933, 'sidewalk': 4934, 'spray': 4935, 'inaccur': 4936, 'curtain': 4937, 'forgiven': 4938, 'meal': 4939, 'porter': 4940, 'comb': 4941, 'miami': 4942, 'kiddi': 4943, 'relentless': 4944, 'goldblum': 4945, 'abound': 4946, 'decapit': 4947, 'carmen': 4948, 'subtli': 4949, 'ash': 4950, 'poker': 4951, 'groan': 4952, 'lauren': 4953, 'walt': 4954, 'basket': 4955, 'galaxi': 4956, 'fruit': 4957, 'miniseri': 4958, 'sacrif': 4959, 'palm': 4960, 'blur': 4961, 'aussi': 4962, 'propheci': 4963, 'orchestr': 4964, 'casper': 4965, 'ingrid': 4966, 'sophi': 4967, 'sparkl': 4968, 'drone': 4969, 'bravo': 4970, 'playboy': 4971, 'hostil': 4972, 'frontal': 4973, 'victorian': 4974, 'motorcycl': 4975, 'masterson': 4976, 'evan': 4977, 'weaker': 4978, 'detach': 4979, 'jill': 4980, 'optimist': 4981, 'vastli': 4982, 'modest': 4983, 'assort': 4984, 'substanti': 4985, 'asset': 4986, 'profil': 4987, 'scariest': 4988, 'incorrect': 4989, 'reincarn': 4990, 'spill': 4991, 'epitom': 4992, 'verg': 4993, 'weari': 4994, 'hackman': 4995, 'cypher': 4996, 'dubiou': 4997, 'scarfac': 4998, 'handicap': 4999}


In [4]:
print(word_dict['movi'])
with open(os.path.join(data_dir, 'word_dict.pkl'), "wb") as f:
    pickle.dump(word_dict, f)


2


### Transform the reviews

Now that we have our word dictionary which allows us to transform the words appearing in the reviews into integers, it is time to make use of it and convert our reviews to their integer sequence representation, making sure to pad or truncate to a fixed length, which in our case is `500`.

In [ ]:
def convert_and_pad(word_dict, sentence, pad=500):
    NOWORD = 0 # We will use 0 to represent the 'no word' category
    INFREQ = 1 # and we use 1 to represent the infrequent words, i.e., words not appearing in word_dict
    
    working_sentence = [NOWORD] * pad
    
    for word_index, word in enumerate(sentence[:pad]):
        if word in word_dict:
            working_sentence[word_index] = word_dict[word]
        else:
            working_sentence[word_index] = INFREQ
            
    return working_sentence, min(len(sentence), pad)

def convert_and_pad_data(word_dict, data, pad=500):
    result = []
    lengths = []
    
    for sentence in data:
        converted, leng = convert_and_pad(word_dict, sentence, pad)
        result.append(converted)
        lengths.append(leng)
        
    return np.array(result), np.array(lengths)

In [ ]:
train_X, train_X_len = convert_and_pad_data(word_dict, train_X)
test_X, test_X_len = convert_and_pad_data(word_dict, test_X)

As a quick check to make sure that things are working as intended, check to see what one of the reviews in the training set looks like after having been processeed. Does this look reasonable? What is the length of a review in the training set?

In [ ]:
# Use this cell to examine one of the processed reviews to make sure everything is working as intended.

**Question:** In the cells above we use the `preprocess_data` and `convert_and_pad_data` methods to process both the training and testing set. Why or why not might this be a problem?

**Answer:**

## Step 3: Upload the data to S3

As in the XGBoost notebook, we will need to upload the training dataset to S3 in order for our training code to access it. For now we will save it locally and we will upload to S3 later on.

### Save the processed training dataset locally

It is important to note the format of the data that we are saving as we will need to know it when we write the training code. In our case, each row of the dataset has the form `label`, `length`, `review[500]` where `review[500]` is a sequence of `500` integers representing the words in the review.

In [ ]:
import pandas as pd
    
pd.concat([pd.DataFrame(train_y), pd.DataFrame(train_X_len), pd.DataFrame(train_X)], axis=1) \
        .to_csv(os.path.join(data_dir, 'train.csv'), header=False, index=False)

### Uploading the training data


Next, we need to upload the training data to the SageMaker default S3 bucket so that we can provide access to it while training our model.

In [ ]:
import sagemaker

sagemaker_session = sagemaker.Session()

bucket = sagemaker_session.default_bucket()
prefix = 'sagemaker/sentiment_rnn'

role = sagemaker.get_execution_role()

In [ ]:
input_data = sagemaker_session.upload_data(path=data_dir, bucket=bucket, key_prefix=prefix)

**NOTE:** The cell above uploads the entire contents of our data directory. This includes the `word_dict.pkl` file. This is fortunate as we will need this later on when we create an endpoint that accepts an arbitrary review. For now, we will just take note of the fact that it resides in the data directory (and so also in the S3 training bucket) and that we will need to make sure it gets saved in the model directory.

## Step 4: Build and Train the PyTorch Model

In the XGBoost notebook we discussed what a model is in the SageMaker framework. In particular, a model comprises three objects

 - Model Artifacts,
 - Training Code, and
 - Inference Code,
 
each of which interact with one another. In the XGBoost example we used training and inference code that was provided by Amazon. Here we will still be using containers provided by Amazon with the added benefit of being able to include our own custom code.

We will start by implementing our own neural network in PyTorch along with a training script. For the purposes of this project we have provided the necessary model object in the `model.py` file, inside of the `train` folder. You can see the provided implementation by running the cell below.

In [ ]:
!pygmentize train/model.py

The important takeaway from the implementation provided is that there are three parameters that we may wish to tweak to improve the performance of our model. These are the embedding dimension, the hidden dimension and the size of the vocabulary. We will likely want to make these parameters configurable in the training script so that if we wish to modify them we do not need to modify the script itself. We will see how to do this later on. To start we will write some of the training code in the notebook so that we can more easily diagnose any issues that arise.

First we will load a small portion of the training data set to use as a sample. It would be very time consuming to try and train the model completely in the notebook as we do not have access to a gpu and the compute instance that we are using is not particularly powerful. However, we can work on a small bit of the data to get a feel for how our training script is behaving.

In [ ]:
import torch
import torch.utils.data

# Read in only the first 250 rows
train_sample = pd.read_csv(os.path.join(data_dir, 'train.csv'), header=None, names=None, nrows=250)

# Turn the input pandas dataframe into tensors
train_sample_y = torch.from_numpy(train_sample[[0]].values).float().squeeze()
train_sample_X = torch.from_numpy(train_sample.drop([0], axis=1).values).long()

# Build the dataset
train_sample_ds = torch.utils.data.TensorDataset(train_sample_X, train_sample_y)
# Build the dataloader
train_sample_dl = torch.utils.data.DataLoader(train_sample_ds, batch_size=50)

### (TODO) Writing the training method

Next we need to write the training code itself. This should be very similar to training methods that you have written before to train PyTorch models. We will leave any difficult aspects such as model saving / loading and parameter loading until a little later.

In [ ]:
def train(model, train_loader, epochs, optimizer, loss_fn, device):
    for epoch in range(1, epochs + 1):
        model.train()
        total_loss = 0
        for batch in train_loader:         
            batch_X, batch_y = batch
            
            batch_X = batch_X.to(device)
            batch_y = batch_y.to(device)
            
            # TODO: Complete this train method to train the model provided.
            
            total_loss += loss.data.item()
        print("Epoch: {}, BCELoss: {}".format(epoch, total_loss / len(train_loader)))

Supposing we have the training method above, we will test that it is working by writing a bit of code in the notebook that executes our training method on the small sample training set that we loaded earlier. The reason for doing this in the notebook is so that we have an opportunity to fix any errors that arise early when they are easier to diagnose.

In [ ]:
import torch.optim as optim
from train.model import LSTMClassifier

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = LSTMClassifier(32, 100, 5000).to(device)
optimizer = optim.Adam(model.parameters())
loss_fn = torch.nn.BCELoss()

train(model, train_sample_dl, 5, optimizer, loss_fn, device)

In order to construct a PyTorch model using SageMaker we must provide SageMaker with a training script. We may optionally include a directory which will be copied to the container and from which our training code will be run. When the training container is executed it will check the uploaded directory (if there is one) for a `requirements.txt` file and install any required Python libraries, after which the training script will be run.

### (TODO) Training the model

When a PyTorch model is constructed in SageMaker, an entry point must be specified. This is the Python file which will be executed when the model is trained. Inside of the `train` directory is a file called `train.py` which has been provided and which contains most of the necessary code to train our model. The only thing that is missing is the implementation of the `train()` method which you wrote earlier in this notebook.

**TODO**: Copy the `train()` method written above and paste it into the `train/train.py` file where required.

The way that SageMaker passes hyperparameters to the training script is by way of arguments. These arguments can then be parsed and used in the training script. To see how this is done take a look at the provided `train/train.py` file.

In [ ]:
from sagemaker.pytorch import PyTorch

estimator = PyTorch(entry_point="train.py",
                    source_dir="train",
                    role=role,
                    framework_version='0.4.0',
                    train_instance_count=1,
                    train_instance_type='ml.p2.xlarge',
                    hyperparameters={
                        'epochs': 10,
                        'hidden_dim': 200,
                    })

In [ ]:
estimator.fit({'training': input_data})

## Step 5: Testing the model

As mentioned at the top of this notebook, we will be testing this model by first deploying it and then sending the testing data to the deployed endpoint. We will do this so that we can make sure that the deployed model is working correctly.

## Step 6: Deploy the model for testing

Now that we have trained our model, we would like to test it to see how it performs. Currently our model takes input of the form `review_length, review[500]` where `review[500]` is a sequence of `500` integers which describe the words present in the review, encoded using `word_dict`. Fortunately for us, SageMaker provides built-in inference code for models with simple inputs such as this.

There is one thing that we need to provide, however, and that is a function which loads the saved model. This function must be called `model_fn()` and takes as its only parameter a path to the directory where the model artifacts are stored. This function must also be present in the python file which we specified as the entry point. In our case the model loading function has been provided and so no changes need to be made.

**NOTE**: When the built-in inference code is run it must import the `model_fn()` method from the `train.py` file. This is why the training code is wrapped in a main guard ( ie, `if __name__ == '__main__':` )

Since we don't need to change anything in the code that was uploaded during training, we can simply deploy the current model as-is.

**NOTE:** When deploying a model you are asking SageMaker to launch an compute instance that will wait for data to be sent to it. As a result, this compute instance will continue to run until *you* shut it down. This is important to know since the cost of a deployed endpoint depends on how long it has been running for.

In other words **If you are no longer using a deployed endpoint, shut it down!**

**TODO:** Deploy the trained model.

In [ ]:
# TODO: Deploy the trained model

## Step 7 - Use the model for testing

Once deployed, we can read in the test data and send it off to our deployed model to get some results. Once we collect all of the results we can determine how accurate our model is.

In [ ]:
test_X = pd.concat([pd.DataFrame(test_X_len), pd.DataFrame(test_X)], axis=1)

In [ ]:
# We split the data into chunks and send each chunk seperately, accumulating the results.

def predict(data, rows=512):
    split_array = np.array_split(data, int(data.shape[0] / float(rows) + 1))
    predictions = np.array([])
    for array in split_array:
        predictions = np.append(predictions, predictor.predict(array))
    
    return predictions

In [ ]:
predictions = predict(test_X.values)
predictions = [round(num) for num in predictions]

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(test_y, predictions)

**Question:** How does this model compare to the XGBoost model you created earlier? Why might these two models perform differently on this dataset? Which do *you* think is better for sentiment analysis?

**Answer:**

### (TODO) More testing

We now have a trained model which has been deployed and which we can send processed reviews to and which returns the predicted sentiment. However, ultimately we would like to be able to send our model an unprocessed review. That is, we would like to send the review itself as a string. For example, suppose we wish to send the following review to our model.

In [ ]:
test_review = 'The simplest pleasures in life are the best, and this film is one of them. Combining a rather basic storyline of love and adventure this movie transcends the usual weekend fair with wit and unmitigated charm.'

The question we now need to answer is, how do we send this review to our model?

Recall in the first section of this notebook we did a bunch of data processing to the IMDb dataset. In particular, we did two specific things to the provided reviews.
 - Removed any html tags and stemmed the input
 - Encoded the review as a sequence of integers using `word_dict`
 
In order process the review we will need to repeat these two steps.

**TODO**: Using the `review_to_words` and `convert_and_pad` methods from section one, convert `test_review` into a numpy array `test_data` suitable to send to our model. Remember that our model expects input of the form `review_length, review[500]`.

In [ ]:
# TODO: Convert test_review into a form usable by the model and save the results in test_data
test_data = None

Now that we have processed the review, we can send the resulting array to our model to predict the sentiment of the review.

In [ ]:
predictor.predict(test_data)

Since the return value of our model is close to `1`, we can be certain that the review we submitted is positive.

### Delete the endpoint

Of course, just like in the XGBoost notebook, once we've deployed an endpoint it continues to run until we tell it to shut down. Since we are done using our endpoint for now, we can delete it.

In [ ]:
estimator.delete_endpoint()

## Step 6 (again) - Deploy the model for the web app

Now that we know that our model is working, it's time to create some custom inference code so that we can send the model a review which has not been processed and have it determine the sentiment of the review.

As we saw above, by default the estimator which we created, when deployed, will use the entry script and directory which we provided when creating the model. However, since we now wish to accept a string as input and our model expects a processed review, we need to write some custom inference code.

We will store the code that we write in the `serve` directory. Provided in this directory is the `model.py` file that we used to construct our model, a `utils.py` file which contains the `review_to_words` and `convert_and_pad` pre-processing functions which we used during the initial data processing, and `predict.py`, the file which will contain our custom inference code. Note also that `requirements.txt` is present which will tell SageMaker what Python libraries are required by our custom inference code.

When deploying a PyTorch model in SageMaker, you are expected to provide four functions which the SageMaker inference container will use.
 - `model_fn`: This function is the same function that we used in the training script and it tells SageMaker how to load our model.
 - `input_fn`: This function receives the raw serialized input that has been sent to the model's endpoint and its job is to de-serialize and make the input available for the inference code.
 - `output_fn`: This function takes the output of the inference code and its job is to serialize this output and return it to the caller of the model's endpoint.
 - `predict_fn`: The heart of the inference script, this is where the actual prediction is done and is the function which you will need to complete.

For the simple website that we are constructing during this project, the `input_fn` and `output_fn` methods are relatively straightforward. We only require being able to accept a string as input and we expect to return a single value as output. You might imagine though that in a more complex application the input or output may be image data or some other binary data which would require some effort to serialize.

### (TODO) Writing inference code

Before writing our custom inference code, we will begin by taking a look at the code which has been provided.

In [ ]:
!pygmentize serve/predict.py

As mentioned earlier, the `model_fn` method is the same as the one provided in the training code and the `input_fn` and `output_fn` methods are very simple and your task will be to complete the `predict_fn` method. Make sure that you save the completed file as `predict.py` in the `serve` directory.

**TODO**: Complete the `predict_fn()` method in the `serve/predict.py` file.

### Deploying the model

Now that the custom inference code has been written, we will create and deploy our model. To begin with, we need to construct a new PyTorchModel object which points to the model artifacts created during training and also points to the inference code that we wish to use. Then we can call the deploy method to launch the deployment container.

**NOTE**: The default behaviour for a deployed PyTorch model is to assume that any input passed to the predictor is a `numpy` array. In our case we want to send a string so we need to construct a simple wrapper around the `RealTimePredictor` class to accomodate simple strings. In a more complicated situation you may want to provide a serialization object, for example if you wanted to sent image data.

In [ ]:
from sagemaker.predictor import RealTimePredictor
from sagemaker.pytorch import PyTorchModel

class StringPredictor(RealTimePredictor):
    def __init__(self, endpoint_name, sagemaker_session):
        super(StringPredictor, self).__init__(endpoint_name, sagemaker_session, content_type='text/plain')

model = PyTorchModel(model_data=estimator.model_data,
                     role = role,
                     framework_version='0.4.0',
                     entry_point='predict.py',
                     source_dir='serve',
                     predictor_cls=StringPredictor)
predictor = model.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge')

### Testing the model

Now that we have deployed our model with the custom inference code, we should test to see if everything is working. Here we test our model by loading the first `250` positive and negative reviews and send them to the endpoint, then collect the results. The reason for only sending some of the data is that the amount of time it takes for our model to process the input and then perform inference is quite long and so testing the entire data set would be prohibitive.

In [ ]:
import glob

def test_reviews(data_dir='../data/aclImdb', stop=250):
    
    results = []
    ground = []
    
    # We make sure to test both positive and negative reviews    
    for sentiment in ['pos', 'neg']:
        
        path = os.path.join(data_dir, 'test', sentiment, '*.txt')
        files = glob.glob(path)
        
        files_read = 0
        
        print('Starting ', sentiment, ' files')
        
        # Iterate through the files and send them to the predictor
        for f in files:
            with open(f) as review:
                # First, we store the ground truth (was the review positive or negative)
                if sentiment == 'pos':
                    ground.append(1)
                else:
                    ground.append(0)
                # Read in the review and convert to 'utf-8' for transmission via HTTP
                review_input = review.read().encode('utf-8')
                # Send the review to the predictor and store the results
                results.append(int(predictor.predict(review_input)))
                
            # Sending reviews to our endpoint one at a time takes a while so we
            # only send a small number of reviews
            files_read += 1
            if files_read == stop:
                break
            
    return ground, results

In [ ]:
ground, results = test_reviews()

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(ground, results)

As an additional test, we can try sending the `test_review` that we looked at earlier.

In [ ]:
predictor.predict(test_review)

Now that we know our endpoint is working as expected, we can set up the web page that will interact with it. If you don't have time to finish the project now, make sure to skip down to the end of this notebook and shut down your endpoint. You can deploy it again when you come back.

## Step 7 (again): Use the model for the web app

> **TODO:** This entire section and the next contain tasks for you to complete, mostly using the AWS console.

So far we have been accessing our model endpoint by constructing a predictor object which uses the endpoint and then just using the predictor object to perform inference. What if we wanted to create a web app which accessed our model? The way things are set up currently makes that not possible since in order to access a SageMaker endpoint the app would first have to authenticate with AWS using an IAM role which included access to SageMaker endpoints. However, there is an easier way! We just need to use some additional AWS services.

<img src="Web App Diagram.svg">

The diagram above gives an overview of how the various services will work together. On the far right is the model which we trained above and which is deployed using SageMaker. On the far left is our web app that collects a user's movie review, sends it off and expects a positive or negative sentiment in return.

In the middle is where some of the magic happens. We will construct a Lambda function, which you can think of as a straightforward Python function that can be executed whenever a specified event occurs. We will give this function permission to send and recieve data from a SageMaker endpoint.

Lastly, the method we will use to execute the Lambda function is a new endpoint that we will create using API Gateway. This endpoint will be a url that listens for data to be sent to it. Once it gets some data it will pass that data on to the Lambda function and then return whatever the Lambda function returns. Essentially it will act as an interface that lets our web app communicate with the Lambda function.

### Setting up a Lambda function

The first thing we are going to do is set up a Lambda function. This Lambda function will be executed whenever our public API has data sent to it. When it is executed it will receive the data, perform any sort of processing that is required, send the data (the review) to the SageMaker endpoint we've created and then return the result.

#### Part A: Create an IAM Role for the Lambda function

Since we want the Lambda function to call a SageMaker endpoint, we need to make sure that it has permission to do so. To do this, we will construct a role that we can later give the Lambda function.

Using the AWS Console, navigate to the **IAM** page and click on **Roles**. Then, click on **Create role**. Make sure that the **AWS service** is the type of trusted entity selected and choose **Lambda** as the service that will use this role, then click **Next: Permissions**.

In the search box type `sagemaker` and select the check box next to the **AmazonSageMakerFullAccess** policy. Then, click on **Next: Review**.

Lastly, give this role a name. Make sure you use a name that you will remember later on, for example `LambdaSageMakerRole`. Then, click on **Create role**.

#### Part B: Create a Lambda function

Now it is time to actually create the Lambda function.

Using the AWS Console, navigate to the AWS Lambda page and click on **Create a function**. When you get to the next page, make sure that **Author from scratch** is selected. Now, name your Lambda function, using a name that you will remember later on, for example `sentiment_analysis_func`. Make sure that the **Python 3.6** runtime is selected and then choose the role that you created in the previous part. Then, click on **Create Function**.

On the next page you will see some information about the Lambda function you've just created. If you scroll down you should see an editor in which you can write the code that will be executed when your Lambda function is triggered. In our example, we will use the code below. 

```python
# We need to use the low-level library to interact with SageMaker since the SageMaker API
# is not available natively through Lambda.
import boto3

def lambda_handler(event, context):

    # The SageMaker runtime is what allows us to invoke the endpoint that we've created.
    runtime = boto3.Session().client('sagemaker-runtime')

    # Now we use the SageMaker runtime to invoke our endpoint, sending the review we were given
    response = runtime.invoke_endpoint(EndpointName = '**ENDPOINT NAME HERE**',    # The name of the endpoint we created
                                       ContentType = 'text/plain',                 # The data format that is expected
                                       Body = event['body'])                       # The actual review

    # The response is an HTTP response whose body contains the result of our inference
    result = response['Body'].read().decode('utf-8')

    return {
        'statusCode' : 200,
        'headers' : { 'Content-Type' : 'text/plain', 'Access-Control-Allow-Origin' : '*' },
        'body' : result
    }
```

Once you have copy and pasted the code above into the Lambda code editor, replace the `**ENDPOINT NAME HERE**` portion with the name of the endpoint that we deployed earlier. You can determine the name of the endpoint using the code cell below.

In [ ]:
predictor.endpoint

Once you have added the endpoint name to the Lambda function, click on **Save**. Your Lambda function is now up and running. Next we need to create a way for our web app to execute the Lambda function.

### Setting up API Gateway

Now that our Lambda function is set up, it is time to create a new API using API Gateway that will trigger the Lambda function we have just created.

Using AWS Console, navigate to **Amazon API Gateway** and then click on **Get started**.

On the next page, make sure that **New API** is selected and give the new api a name, for example, `sentiment_analysis_api`. Then, click on **Create API**.

Now we have created an API, however it doesn't currently do anything. What we want it to do is to trigger the Lambda function that we created earlier.

Select the **Actions** dropdown menu and click **Create Method**. A new blank method will be created, select its dropdown menu and select **POST**, then click on the check mark beside it.

For the integration point, make sure that **Lambda Function** is selected and click on the **Use Lambda Proxy integration**. This option makes sure that the data that is sent to the API is then sent directly to the Lambda function with no processing. It also means that the return value must be a proper response object as it will also not be processed by API Gateway.

Type the name of the Lambda function you created earlier into the **Lambda Function** text entry box and then click on **Save**. Click on **OK** in the pop-up box that then appears, giving permission to API Gateway to invoke the Lambda function you created.

The last step in creating the API Gateway is to select the **Actions** dropdown and click on **Deploy API**. You will need to create a new Deployment stage and name it anything you like, for example `prod`.

You have now successfully set up a public API to access your SageMaker model. Make sure to copy or write down the URL provided to invoke your newly created public API as this will be needed in the next step. This URL can be found at the top of the page, highlighted in blue next to the text **Invoke URL**.

## Step 4: Deploying our web app

Now that we have a publicly available API, we can start using it in a web app. For our purposes, we have provided a simple static html file which can make use of the public api you created earlier.

In the `website` folder there should be a file called `index.html`. Download the file to your computer and open that file up in a text editor of your choice. There should be a line which contains **\*\*REPLACE WITH PUBLIC API URL\*\***. Replace this string with the url that you wrote down in the last step and then save the file.

Now, if you open `index.html` on your local computer, your browser will behave as a local web server and you can use the provided site to interact with your SageMaker model.

If you'd like to go further, you can host this html file anywhere you'd like, for example using github or hosting a static site on Amazon's S3. Once you have done this you can share the link with anyone you'd like and have them play with it too!

> **Important Note** In order for the web app to communicate with the SageMaker endpoint, the endpoint has to actually be deployed and running. This means that you are paying for it. Make sure that the endpoint is running when you want to use the web app but that you shut it down when you don't need it, otherwise you will end up with a surprisingly large AWS bill.

**TODO:** Make sure that you include the edited `index.html` file in your project submission.

Now that your web app is working, trying playing around with it and see how well it works.

**Question**: Give an example of a review that you entered into your web app. What was the predicted sentiment of your example review?

**Answer:**

### Delete the endpoint

Remember to always shut down your endpoint if you are no longer using it. You are charged for the length of time that the endpoint is running so if you forget and leave it on you could end up with an unexpectedly large bill.

In [ ]:
predictor.delete_endpoint()